# 構造研ミーティング用ケース

|      | 1年目 | 2年目 | 3年目 | 4年目 | 5年目 |
| ---- | ---- | ---- | ---- | ---- | ---- |
| 設置海域I1 | 5 | 年 | で | 35 | 基 |
| 設置海域I2 | 5 | 年 | で | 35 | 基 |
| 設置海域I3 | 5 | 年 | で | 35 | 基 |
| 設置海域I4 | 5 | 年 | で | 35 | 基 |

### ノードとリンクを設定

In [158]:
# ノード用
n_taple = ("00", "01", "02", "03", "04", "05", "06", "07", "08", "09",
           "10", "11", "12", "13", "14", "15", "16", "17", "18", "19")

# リンク用   base    base    base    base    green  install install install install
l_taple = ("0003", "0004", "0005", "0006", "0007", "0008", "0009", "0010", "0011",
           "0103", "0104", "0105", "0106", "0107", "0108", "0109", "0110", "0111",
           "0203", "0204", "0205", "0206", "0207", "0208", "0209", "0210", "0211",
                                                   "0308", "0309", "0310", "0311",
                                                   "0408", "0409", "0410", "0411",
                                                   "0508", "0509", "0510", "0511",
                                                   "0608", "0609", "0610", "0611",
                                                   "0708", "0709", "0710", "0711",
           "1203", "1204", "1205", "1206", "1207", "1208", "1209", "1210", "1211",
           "1303", "1304", "1305", "1306", "1307", "1308", "1309", "1310", "1311",
           "1403", "1404", "1405", "1406", "1407", "1408", "1409", "1410", "1411",
           "1503", "1504", "1505", "1506", "1507", "1508", "1509", "1510", "1511",
           "1603",                                 "1608",
                   "1704",                                 "1709",
                           "1805",                                 "1810",
                                   "1906",                                 "1911",
           "0303", "0404", "0505", "0606",         "0808", "0909", "1010", "1111")

### 入力データ集

#### 風車の組立コスト $cx$ ・リンク使用コスト $cs_{use}$ ・リンク組立コスト $cs_{start}$

In [ ]:
## 組立コスト
# ベースポートでの風車組立コスト
cxb = 5
# 設置海域での風車組立コスト
cxs = 10

## リンクにかかるコスト
# リンク使用コスト
csu = 5
# リンク建設コスト
css = 15

#### キャパシティに関する定数 $u$

輸送船の輸送上限数（浮体基礎を輸送するのは別の数としている）

In [160]:
# 輸送船の輸送上限数（浮体基礎以外）
uy = 3    # 1年目
uy2 = 4   # 2年目
uy3 = 5   # 3年目
uy4 = 6   # 4年目
uy5 = 7   # 5年目

# 輸送船の輸送上限数（浮体基礎）
uy_ = 2    # 1年目
uy2_ = 3   # 2年目
uy3_ = 4   # 3年目
uy4_ = 5   # 4年目
uy5_ = 6   # 5年目

設置船の輸送上限数

In [161]:
# 設置船の輸送上限数
us = 3    # 1年目
us2 = 4   # 2年目
us3 = 5   # 3年目
us4 = 6   # 4年目
us5 = 7   # 5年目

ベースポートでの組立上限

In [162]:
# ベースポートでの組立数上限
ub = 3   # 1年目
ub2 = 4  # 2年目
ub3 = 5  # 3年目
ub4 = 6  # 4年目
ub5 = 7  # 5年目

設置海域での組立上限

In [163]:
# 設置海域での組立数上限
usa = 2   # 1年目
usa2 = 3  # 2年目
usa3 = 4  # 3年目
usa4 = 5  # 4年目
usa5 = 6  # 5年目

### 以下メイン部分

#### ライブラリの導入

In [164]:
import numpy as np
from pyscipopt import Model, quicksum

#### モデル作成、年数・ノード数・リンク数の設定

In [165]:
# モデルを作成
model = Model("20240726")

# 10年間の計画
# ここは入力
N_network = 5

# ノード数、リンク数、製品数
num_node = len(n_taple)     # number of nodes = 20
num_link = len(l_taple)     # number of links = 99
num_product = 5             # number of components and products

In [166]:
#### 変数の設定

In [167]:
# 輸送量の変数
x = {}
for m in range(0, N_network):
    for i in range(0, num_link):
        for j in range(0, num_product):
            for k in range(0, 2):
                x[m,i,j,k] = model.addVar(vtype="I", name="x[%s,%s,%s,%s]" % (m, i, j, k))

# リンク使用コストを計算するための変数
s = {}
for m in range(0, N_network):
    for i in range(0, num_link):
        for k in range(0, 2):
            s[m,i,k] = model.addVar(vtype="B", name="s_use[%s,%s,%s]" % (m, i, k))

# 設置海域の風車需要量を変数で表す、insは設置海域の数
matome = {}
for ins in range(4):
    for m in range(0, N_network):
        matome[ins,m] = model.addVar(vtype="I", name="matome[%s,%s]" % (ins, m))

#### パラメータ・入力値の設定

ノードとリンクの接続関係を表す $A$ 行列

$A$ 行列の設定（流出）

In [168]:
# 流出
A_plus =  [m for m in range(N_network)] 

node_num = len(n_taple)
link_num = len(l_taple)

# A_plus番号を入れる関数
def An_idx(org_n_idx: str, dst_n_index: str):
    l_idx = org_n_idx + dst_n_index
    return n_taple.index(org_n_idx), l_taple.index(l_idx)

# 1年目
A_plus[0] = np.zeros((node_num, link_num))

A_plus[0][An_idx("00", "03")] = 1
A_plus[0][An_idx("00", "04")] = 1
A_plus[0][An_idx("00", "05")] = 1
A_plus[0][An_idx("00", "06")] = 1
A_plus[0][An_idx("00", "07")] = 1
A_plus[0][An_idx("00", "08")] = 1
A_plus[0][An_idx("00", "09")] = 1
A_plus[0][An_idx("00", "10")] = 1
A_plus[0][An_idx("00", "11")] = 1
A_plus[0][An_idx("01", "03")] = 1
A_plus[0][An_idx("01", "04")] = 1
A_plus[0][An_idx("01", "05")] = 1
A_plus[0][An_idx("01", "06")] = 1
A_plus[0][An_idx("01", "07")] = 1
A_plus[0][An_idx("01", "08")] = 1
A_plus[0][An_idx("01", "09")] = 1
A_plus[0][An_idx("01", "10")] = 1
A_plus[0][An_idx("01", "11")] = 1
A_plus[0][An_idx("02", "03")] = 1
A_plus[0][An_idx("02", "04")] = 1
A_plus[0][An_idx("02", "05")] = 1
A_plus[0][An_idx("02", "06")] = 1
A_plus[0][An_idx("02", "07")] = 1
A_plus[0][An_idx("02", "08")] = 1
A_plus[0][An_idx("02", "09")] = 1
A_plus[0][An_idx("02", "10")] = 1
A_plus[0][An_idx("02", "11")] = 1
A_plus[0][An_idx("03", "08")] = 1
A_plus[0][An_idx("03", "09")] = 1
A_plus[0][An_idx("03", "10")] = 1
A_plus[0][An_idx("03", "11")] = 1
A_plus[0][An_idx("04", "08")] = 1
A_plus[0][An_idx("04", "09")] = 1
A_plus[0][An_idx("04", "10")] = 1
A_plus[0][An_idx("04", "11")] = 1
A_plus[0][An_idx("05", "08")] = 1
A_plus[0][An_idx("05", "09")] = 1
A_plus[0][An_idx("05", "10")] = 1
A_plus[0][An_idx("05", "11")] = 1
A_plus[0][An_idx("06", "08")] = 1
A_plus[0][An_idx("06", "09")] = 1
A_plus[0][An_idx("06", "10")] = 1
A_plus[0][An_idx("06", "11")] = 1
A_plus[0][An_idx("07", "08")] = 1
A_plus[0][An_idx("07", "09")] = 1
A_plus[0][An_idx("07", "10")] = 1
A_plus[0][An_idx("07", "11")] = 1
A_plus[0][An_idx("12", "03")] = 1
A_plus[0][An_idx("12", "04")] = 1
A_plus[0][An_idx("12", "05")] = 1
A_plus[0][An_idx("12", "06")] = 1
A_plus[0][An_idx("12", "07")] = 1
A_plus[0][An_idx("12", "08")] = 1
A_plus[0][An_idx("12", "09")] = 1
A_plus[0][An_idx("12", "10")] = 1
A_plus[0][An_idx("12", "11")] = 1
A_plus[0][An_idx("13", "03")] = 1
A_plus[0][An_idx("13", "04")] = 1
A_plus[0][An_idx("13", "05")] = 1
A_plus[0][An_idx("13", "06")] = 1
A_plus[0][An_idx("13", "07")] = 1
A_plus[0][An_idx("13", "08")] = 1
A_plus[0][An_idx("13", "09")] = 1
A_plus[0][An_idx("13", "10")] = 1
A_plus[0][An_idx("13", "11")] = 1
A_plus[0][An_idx("14", "03")] = 1
A_plus[0][An_idx("14", "04")] = 1
A_plus[0][An_idx("14", "05")] = 1
A_plus[0][An_idx("14", "06")] = 1
A_plus[0][An_idx("14", "07")] = 1
A_plus[0][An_idx("14", "08")] = 1
A_plus[0][An_idx("14", "09")] = 1
A_plus[0][An_idx("14", "10")] = 1
A_plus[0][An_idx("14", "11")] = 1
A_plus[0][An_idx("15", "03")] = 1
A_plus[0][An_idx("15", "04")] = 1
A_plus[0][An_idx("15", "05")] = 1
A_plus[0][An_idx("15", "06")] = 1
A_plus[0][An_idx("15", "07")] = 1
A_plus[0][An_idx("15", "08")] = 1
A_plus[0][An_idx("15", "09")] = 1
A_plus[0][An_idx("15", "10")] = 1
A_plus[0][An_idx("15", "11")] = 1
A_plus[0][An_idx("16", "03")] = 1
A_plus[0][An_idx("16", "08")] = 1
A_plus[0][An_idx("17", "04")] = 1
A_plus[0][An_idx("17", "09")] = 1
A_plus[0][An_idx("18", "05")] = 1
A_plus[0][An_idx("18", "10")] = 1
A_plus[0][An_idx("19", "06")] = 1
A_plus[0][An_idx("19", "11")] = 1
A_plus[0][An_idx("03", "03")] = 1
A_plus[0][An_idx("04", "04")] = 1
A_plus[0][An_idx("05", "05")] = 1
A_plus[0][An_idx("06", "06")] = 1
A_plus[0][An_idx("08", "08")] = 1
A_plus[0][An_idx("09", "09")] = 1
A_plus[0][An_idx("10", "10")] = 1
A_plus[0][An_idx("11", "11")] = 1

# 2年目以降
for m in range(1, N_network):
    A_plus[m] = A_plus[0].copy()

$A$ 行列の設定（流入）

In [169]:
# 流入
A_minus =  [m for m in range(N_network)] 

# A_minus番号を入れる関数
def An_minus_idx(org_n_idx: str, dst_n_index: str):
    l_idx = org_n_idx + dst_n_index
    return n_taple.index(dst_n_index), l_taple.index(l_idx)

# 1年目
A_minus[0] = np.zeros((node_num, link_num))

A_minus[0][An_minus_idx("00", "03")] = 1
A_minus[0][An_minus_idx("00", "04")] = 1
A_minus[0][An_minus_idx("00", "05")] = 1
A_minus[0][An_minus_idx("00", "06")] = 1
A_minus[0][An_minus_idx("00", "07")] = 1
A_minus[0][An_minus_idx("00", "08")] = 1
A_minus[0][An_minus_idx("00", "09")] = 1
A_minus[0][An_minus_idx("00", "10")] = 1
A_minus[0][An_minus_idx("00", "11")] = 1
A_minus[0][An_minus_idx("01", "03")] = 1
A_minus[0][An_minus_idx("01", "04")] = 1
A_minus[0][An_minus_idx("01", "05")] = 1
A_minus[0][An_minus_idx("01", "06")] = 1
A_minus[0][An_minus_idx("01", "07")] = 1
A_minus[0][An_minus_idx("01", "08")] = 1
A_minus[0][An_minus_idx("01", "09")] = 1
A_minus[0][An_minus_idx("01", "10")] = 1
A_minus[0][An_minus_idx("01", "11")] = 1
A_minus[0][An_minus_idx("02", "03")] = 1
A_minus[0][An_minus_idx("02", "04")] = 1
A_minus[0][An_minus_idx("02", "05")] = 1
A_minus[0][An_minus_idx("02", "06")] = 1
A_minus[0][An_minus_idx("02", "07")] = 1
A_minus[0][An_minus_idx("02", "08")] = 1
A_minus[0][An_minus_idx("02", "09")] = 1
A_minus[0][An_minus_idx("02", "10")] = 1
A_minus[0][An_minus_idx("02", "11")] = 1
A_minus[0][An_minus_idx("03", "08")] = 1
A_minus[0][An_minus_idx("03", "09")] = 1
A_minus[0][An_minus_idx("03", "10")] = 1
A_minus[0][An_minus_idx("03", "11")] = 1
A_minus[0][An_minus_idx("04", "08")] = 1
A_minus[0][An_minus_idx("04", "09")] = 1
A_minus[0][An_minus_idx("04", "10")] = 1
A_minus[0][An_minus_idx("04", "11")] = 1
A_minus[0][An_minus_idx("05", "08")] = 1
A_minus[0][An_minus_idx("05", "09")] = 1
A_minus[0][An_minus_idx("05", "10")] = 1
A_minus[0][An_minus_idx("05", "11")] = 1
A_minus[0][An_minus_idx("06", "08")] = 1
A_minus[0][An_minus_idx("06", "09")] = 1
A_minus[0][An_minus_idx("06", "10")] = 1
A_minus[0][An_minus_idx("06", "11")] = 1
A_minus[0][An_minus_idx("07", "08")] = 1
A_minus[0][An_minus_idx("07", "09")] = 1
A_minus[0][An_minus_idx("07", "10")] = 1
A_minus[0][An_minus_idx("07", "11")] = 1
A_minus[0][An_minus_idx("12", "03")] = 1
A_minus[0][An_minus_idx("12", "04")] = 1
A_minus[0][An_minus_idx("12", "05")] = 1
A_minus[0][An_minus_idx("12", "06")] = 1
A_minus[0][An_minus_idx("12", "07")] = 1
A_minus[0][An_minus_idx("12", "08")] = 1
A_minus[0][An_minus_idx("12", "09")] = 1
A_minus[0][An_minus_idx("12", "10")] = 1
A_minus[0][An_minus_idx("12", "11")] = 1
A_minus[0][An_minus_idx("13", "03")] = 1
A_minus[0][An_minus_idx("13", "04")] = 1
A_minus[0][An_minus_idx("13", "05")] = 1
A_minus[0][An_minus_idx("13", "06")] = 1
A_minus[0][An_minus_idx("13", "07")] = 1
A_minus[0][An_minus_idx("13", "08")] = 1
A_minus[0][An_minus_idx("13", "09")] = 1
A_minus[0][An_minus_idx("13", "10")] = 1
A_minus[0][An_minus_idx("13", "11")] = 1
A_minus[0][An_minus_idx("14", "03")] = 1
A_minus[0][An_minus_idx("14", "04")] = 1
A_minus[0][An_minus_idx("14", "05")] = 1
A_minus[0][An_minus_idx("14", "06")] = 1
A_minus[0][An_minus_idx("14", "07")] = 1
A_minus[0][An_minus_idx("14", "08")] = 1
A_minus[0][An_minus_idx("14", "09")] = 1
A_minus[0][An_minus_idx("14", "10")] = 1
A_minus[0][An_minus_idx("14", "11")] = 1
A_minus[0][An_minus_idx("15", "03")] = 1
A_minus[0][An_minus_idx("15", "04")] = 1
A_minus[0][An_minus_idx("15", "05")] = 1
A_minus[0][An_minus_idx("15", "06")] = 1
A_minus[0][An_minus_idx("15", "07")] = 1
A_minus[0][An_minus_idx("15", "08")] = 1
A_minus[0][An_minus_idx("15", "09")] = 1
A_minus[0][An_minus_idx("15", "10")] = 1
A_minus[0][An_minus_idx("15", "11")] = 1
A_minus[0][An_minus_idx("16", "03")] = 1
A_minus[0][An_minus_idx("16", "08")] = 1
A_minus[0][An_minus_idx("17", "04")] = 1
A_minus[0][An_minus_idx("17", "09")] = 1
A_minus[0][An_minus_idx("18", "05")] = 1
A_minus[0][An_minus_idx("18", "10")] = 1
A_minus[0][An_minus_idx("19", "06")] = 1
A_minus[0][An_minus_idx("19", "11")] = 1
A_minus[0][An_minus_idx("03", "03")] = 1
A_minus[0][An_minus_idx("04", "04")] = 1
A_minus[0][An_minus_idx("05", "05")] = 1
A_minus[0][An_minus_idx("06", "06")] = 1
A_minus[0][An_minus_idx("08", "08")] = 1
A_minus[0][An_minus_idx("09", "09")] = 1
A_minus[0][An_minus_idx("10", "10")] = 1
A_minus[0][An_minus_idx("11", "11")] = 1

# 2年目以降
for m in range(1, N_network):
    A_minus[m] = A_minus[0].copy()

風車の需給量を表す $b$ 行列

In [170]:
# 風車の需給量の設定
b = [m for m in range(N_network)]

# 1年目
b[0] = np.array([
    # node
    # tower, blade, nacelle, foundation, product
    [  0,  0,999,  0,  0], # Node0、横浜（生産拠点、ナセル）
    [  0,  0,  0,999,  0], # Node1、津（生産拠点、浮体基礎）
    [999,  0,  0,  0,  0], # Node2、神戸（生産拠点、タワー）

    [  0,  0,  0,  0,  0], # Node3、石狩湾新港（ベースポート）
    [  0,  0,  0,  0,  0], # Node4、鹿島港（ベースポート）
    [  0,  0,  0,  0,  0], # Node5、北九州港（ベースポート）
    [  0,  0,  0,  0,  0], # Node6、秋田港（ベースポート）

    [  0,  0,  0,  0,  0], # Node7、能代港（部材積替地点）

    [  0,  0,  0,  0, -matome[0, 0]], # Node8、島牧沖（設置海域）
    [  0,  0,  0,  0, -matome[1, 0]], # Node9、銚子沖（設置海域）
    [  0,  0,  0,  0, -matome[2, 0]], # Node10、五島市沖（設置海域）
    [  0,  0,  0,  0, -matome[3, 0]], # Node11、能代沖（設置海域）

    [999,  0,  0,  0,  0], # Node12、石狩湾新港（生産拠点候補、タワー）
    [  0,  0,  0,999,  0], # Node13、鹿島港（生産拠点候補、浮体基礎）
    [  0,  0,999,  0,  0], # Node14、北九州港（生産拠点候補、ナセル）
    [  0,999,  0,  0,  0], # Node15、秋田港（生産拠点候補、ブレード）

# 輸入(このケースでは使用してないです)
    [  0,  0,  0,  0,  0], # Node16、石狩湾新港（ブレード輸入）
    [  0,  0,  0,  0,  0], # Node17、鹿島港（ブレード輸入）
    [  0,  0,  0,  0,  0], # Node18、北九州港（ブレード輸入）
    [  0,  0,  0,  0,  0], # Node19、秋田港（ブレード輸入）
])

# 2年目以降
# 行列の大枠を決定
for m in range(1, N_network):
    b[m] = b[0].copy()
    # m年目の各設置海域需要量を変数として設定
    for ins in range(8, 12):
        b[m][ins, 4] = -matome[ins-8, m]

変数 $matome$ は行が設置海域番号を、列が年数を表す

フローにかかるコスト $cx$ 輸送コストと組立コスト

In [171]:
# フローにかかるコスト
c_x = [m for m in range(N_network)]

# 1年目
c_x[0] = np.array([
    # link
    # tower, blade, nacelle, foundation, product,
    # outflow, inflow
    [[   0,   0],[   0,   0],[ 7.1,   0],[   0,   0],[   0,   0]], # link0、0→3
    [[   0,   0],[   0,   0],[ 1.3,   0],[   0,   0],[   0,   0]], # link1、0→4
    [[   0,   0],[   0,   0],[ 5.3,   0],[   0,   0],[   0,   0]], # link2、0→5
    [[   0,   0],[   0,   0],[ 6.4,   0],[   0,   0],[   0,   0]], # link3、0→6
    [[   0,   0],[   0,   0],[ 6.0,   0],[   0,   0],[   0,   0]], # link4、0→7
    [[   0,   0],[   0,   0],[ 6.7,   0],[   0,   0],[   0,   0]], # link5、0→8
    [[   0,   0],[   0,   0],[ 1.2,   0],[   0,   0],[   0,   0]], # link6、0→9
    [[   0,   0],[   0,   0],[ 7.0,   0],[   0,   0],[   0,   0]], # link7、0→10
    [[   0,   0],[   0,   0],[ 6.3,   0],[   0,   0],[   0,   0]], # link8、0→11
    [[   0,   0],[   0,   0],[   0,   0],[ 8.3,   0],[   0,   0]], # link9、1→3
    [[   0,   0],[   0,   0],[   0,   0],[ 2.6,   0],[   0,   0]], # link10、1→4
    [[   0,   0],[   0,   0],[   0,   0],[ 4.0,   0],[   0,   0]], # link11、1→5
    [[   0,   0],[   0,   0],[   0,   0],[ 7.7,   0],[   0,   0]], # link12、1→6
    [[   0,   0],[   0,   0],[   0,   0],[ 7.3,   0],[   0,   0]], # link13、1→7
    [[   0,   0],[   0,   0],[   0,   0],[ 8.0,   0],[   0,   0]], # link14、1→8
    [[   0,   0],[   0,   0],[   0,   0],[ 2.5,   0],[   0,   0]], # link15、1→9
    [[   0,   0],[   0,   0],[   0,   0],[ 5.4,   0],[   0,   0]], # link16、1→10
    [[   0,   0],[   0,   0],[   0,   0],[ 7.6,   0],[   0,   0]], # link17、1→11
    [[ 9.9,   0],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link18、2→3
    [[ 4.1,   0],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link19、2→4
    [[ 3.2,   0],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link20、2→5
    [[ 9.2,   0],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link21、2→6
    [[ 8.8,   0],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link22、2→7
    [[ 9.6,   0],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link23、2→8
    [[ 4.0,   0],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link24、2→9
    [[ 3.7,   0],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link25、2→10
    [[ 7.9,   0],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link26、2→11

    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[ 9.7,   0]], # link27、3→8
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[74.9,   0]], # link28、3→9
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[ 107,   0]], # link29、3→10
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[32.9,   0]], # link30、3→11
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[66.7,   0]], # link31、4→8
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[ 7.1,   0]], # link32、4→9
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[87.3,   0]], # link33、4→10
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[60.6,   0]], # link34、4→11
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[84.6,   0]], # link35、5→8
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[70.0,   0]], # link36、5→9
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[16.6,   0]], # link37、5→10
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[67.3,   0]], # link38、5→11
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[27.3,   0]], # link39、6→8
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[66.3,   0]], # link40、6→9
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[83.4,   0]], # link41、6→10
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[ 3.0,   0]], # link42、6→11
    [[ 3.8,   0],[ 3.8,   0],[ 3.8,   0],[ 3.8,   0],[   0,   0]], # link43、7→8
    [[10.3,   0],[10.3,   0],[10.3,   0],[10.3,   0],[   0,   0]], # link44、7→9
    [[14.1,   0],[14.1,   0],[14.1,   0],[14.1,   0],[   0,   0]], # link45、7→10
    [[ 0.7,   0],[ 0.7,   0],[ 0.7,   0],[ 0.7,   0],[   0,   0]], # link46、7→11

    [[0.0001,   0],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link47、12→3
    [[ 5.8,   0],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link48、12→4
    [[ 7.7,   0],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link49、12→5
    [[ 2.8,   0],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link50、12→6
    [[ 2.4,   0],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link51、12→7
    [[ 0.8,   0],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link52、12→8
    [[ 6.2,   0],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link53、12→9
    [[ 8.9,   0],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link54、12→10
    [[ 2.7,   0],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link55、12→11   
    [[   0,   0],[   0,   0],[   0,   0],[ 5.8,   0],[   0,   0]], # link56、13→3
    [[   0,   0],[   0,   0],[   0,   0],[0.0001,   0],[   0,   0]], # link57、13→4
    [[   0,   0],[   0,   0],[   0,   0],[ 5.8,   0],[   0,   0]], # link58、13→5
    [[   0,   0],[   0,   0],[   0,   0],[ 5.2,   0],[   0,   0]], # link59、13→6
    [[   0,   0],[   0,   0],[   0,   0],[ 4.7,   0],[   0,   0]], # link60、13→7
    [[   0,   0],[   0,   0],[   0,   0],[ 5.6,   0],[   0,   0]], # link61、13→8
    [[   0,   0],[   0,   0],[   0,   0],[ 0.6,   0],[   0,   0]], # link62、13→9
    [[   0,   0],[   0,   0],[   0,   0],[ 7.3,   0],[   0,   0]], # link63、13→10
    [[   0,   0],[   0,   0],[   0,   0],[ 5.0,   0],[   0,   0]], # link64、13→11
    [[   0,   0],[   0,   0],[ 7.7,   0],[   0,   0],[   0,   0]], # link65、14→3
    [[   0,   0],[   0,   0],[ 5.8,   0],[   0,   0],[   0,   0]], # link66、14→4
    [[   0,   0],[   0,   0],[0.0001,   0],[   0,   0],[   0,   0]], # link67、14→5
    [[   0,   0],[   0,   0],[ 5.9,   0],[   0,   0],[   0,   0]], # link68、14→6
    [[   0,   0],[   0,   0],[ 6.0,   0],[   0,   0],[   0,   0]], # link69、14→7
    [[   0,   0],[   0,   0],[ 7.1,   0],[   0,   0],[   0,   0]], # link70、14→8
    [[   0,   0],[   0,   0],[ 5.8,   0],[   0,   0],[   0,   0]], # link71、14→9
    [[   0,   0],[   0,   0],[ 1.4,   0],[   0,   0],[   0,   0]], # link72、14→10
    [[   0,   0],[   0,   0],[ 5.6,   0],[   0,   0],[   0,   0]], # link73、14→11
    [[   0,   0],[ 2.8,   0],[   0,   0],[   0,   0],[   0,   0]], # link74、15→3
    [[   0,   0],[ 5.2,   0],[   0,   0],[   0,   0],[   0,   0]], # link75、15→4
    [[   0,   0],[ 5.9,   0],[   0,   0],[   0,   0],[   0,   0]], # link76、15→5
    [[   0,   0],[0.001,   0],[   0,   0],[   0,   0],[   0,   0]], # link77、15→6
    [[   0,   0],[ 0.4,   0],[   0,   0],[   0,   0],[   0,   0]], # link78、15→7
    [[   0,   0],[ 2.3,   0],[   0,   0],[   0,   0],[   0,   0]], # link79、15→8
    [[   0,   0],[ 5.5,   0],[   0,   0],[   0,   0],[   0,   0]], # link80、15→9
    [[   0,   0],[ 7.0,   0],[   0,   0],[   0,   0],[   0,   0]], # link81、15→10
    [[   0,   0],[ 0.3,   0],[   0,   0],[   0,   0],[   0,   0]], # link82、15→11

    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link83、16→3
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link84、16→8
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link85、17→4
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link86、17→9
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link87、18→5
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link88、18→10
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link89、19→6
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link90、19→11

    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[   0, cxb]], # link91、3→3
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[   0, cxb]], # link92、4→4
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[   0, cxb]], # link93、5→5
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[   0, cxb]], # link94、6→6
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[   0, cxs]], # link95、8→8
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[   0, cxs]], # link96、9→9
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[   0, cxs]], # link97、10→10
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[   0, cxs]], # link98、11→11
])

# 2年目以降
for m in range(1, N_network):
    c_x[m] = c_x[0].copy()

変数の確認

$cxb$ はベースポートでの組立コスト  
$cxs$ は設置海域での組立コスト
<br>

$csu$ はリンク使用コスト  
$css$はリンク建設コスト
<br><br><br>

$uy$ 輸送船の輸送キャパシティ（浮体基礎以外）  
$uy\_$ 輸送船の輸送キャパシティ（浮体基礎） 
$us$ 設置船の輸送キャパシティ  
$ub$ ベースポートの組立キャパシティ  
$usa$ 設置海域の組立キャパシティ

In [172]:
c_s_use = [m for m in range(N_network)]

# 1年目、生産拠点から流出するリンクにかかる
c_s_use[0] = np.array([
    # link
    # tower, blade, nacelle, foundation, product,
    # outflow, inflow
    [[ csu,   0],[ csu,   0],[ csu,   0],[ csu,   0],[   0,   0]], # link0、0→3
    [[ csu,   0],[ csu,   0],[ csu,   0],[ csu,   0],[   0,   0]], # link1、0→4
    [[ csu,   0],[ csu,   0],[ csu,   0],[ csu,   0],[   0,   0]], # link2、0→5
    [[ csu,   0],[ csu,   0],[ csu,   0],[ csu,   0],[   0,   0]], # link3、0→6
    [[ csu,   0],[ csu,   0],[ csu,   0],[ csu,   0],[   0,   0]], # link4、0→7
    [[ csu,   0],[ csu,   0],[ csu,   0],[ csu,   0],[   0,   0]], # link5、0→8
    [[ csu,   0],[ csu,   0],[ csu,   0],[ csu,   0],[   0,   0]], # link6、0→9
    [[ csu,   0],[ csu,   0],[ csu,   0],[ csu,   0],[   0,   0]], # link7、0→10
    [[ csu,   0],[ csu,   0],[ csu,   0],[ csu,   0],[   0,   0]], # link8、0→11
    [[ csu,   0],[ csu,   0],[ csu,   0],[ csu,   0],[   0,   0]], # link9、1→3
    [[ csu,   0],[ csu,   0],[ csu,   0],[ csu,   0],[   0,   0]], # link10、1→4
    [[ csu,   0],[ csu,   0],[ csu,   0],[ csu,   0],[   0,   0]], # link11、1→5
    [[ csu,   0],[ csu,   0],[ csu,   0],[ csu,   0],[   0,   0]], # link12、1→6
    [[ csu,   0],[ csu,   0],[ csu,   0],[ csu,   0],[   0,   0]], # link13、1→7
    [[ csu,   0],[ csu,   0],[ csu,   0],[ csu,   0],[   0,   0]], # link14、1→8
    [[ csu,   0],[ csu,   0],[ csu,   0],[ csu,   0],[   0,   0]], # link15、1→9
    [[ csu,   0],[ csu,   0],[ csu,   0],[ csu,   0],[   0,   0]], # link16、1→10
    [[ csu,   0],[ csu,   0],[ csu,   0],[ csu,   0],[   0,   0]], # link17、1→11
    [[ csu,   0],[ csu,   0],[ csu,   0],[ csu,   0],[   0,   0]], # link18、2→3
    [[ csu,   0],[ csu,   0],[ csu,   0],[ csu,   0],[   0,   0]], # link19、2→4
    [[ csu,   0],[ csu,   0],[ csu,   0],[ csu,   0],[   0,   0]], # link20、2→5
    [[ csu,   0],[ csu,   0],[ csu,   0],[ csu,   0],[   0,   0]], # link21、2→6
    [[ csu,   0],[ csu,   0],[ csu,   0],[ csu,   0],[   0,   0]], # link22、2→7
    [[ csu,   0],[ csu,   0],[ csu,   0],[ csu,   0],[   0,   0]], # link23、2→8
    [[ csu,   0],[ csu,   0],[ csu,   0],[ csu,   0],[   0,   0]], # link24、2→9
    [[ csu,   0],[ csu,   0],[ csu,   0],[ csu,   0],[   0,   0]], # link25、2→10
    [[ csu,   0],[ csu,   0],[ csu,   0],[ csu,   0],[   0,   0]], # link26、2→11

    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link27、3→8
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link28、3→9
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link29、3→10
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link30、3→11
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link31、4→8
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link32、4→9
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link33、4→10
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link34、4→11
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link35、5→8
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link36、5→9
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link37、5→10
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link38、5→11
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link39、6→8
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link40、6→9
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link41、6→10
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link42、6→11
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link43、7→8
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link44、7→9
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link45、7→10
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link46、7→11

    [[ csu,   0],[ csu,   0],[ csu,   0],[ csu,   0],[   0,   0]], # link47、12→3
    [[ csu,   0],[ csu,   0],[ csu,   0],[ csu,   0],[   0,   0]], # link48、12→4
    [[ csu,   0],[ csu,   0],[ csu,   0],[ csu,   0],[   0,   0]], # link49、12→5
    [[ csu,   0],[ csu,   0],[ csu,   0],[ csu,   0],[   0,   0]], # link50、12→6
    [[ csu,   0],[ csu,   0],[ csu,   0],[ csu,   0],[   0,   0]], # link51、12→7
    [[ csu,   0],[ csu,   0],[ csu,   0],[ csu,   0],[   0,   0]], # link52、12→8
    [[ csu,   0],[ csu,   0],[ csu,   0],[ csu,   0],[   0,   0]], # link53、12→9
    [[ csu,   0],[ csu,   0],[ csu,   0],[ csu,   0],[   0,   0]], # link54、12→10
    [[ csu,   0],[ csu,   0],[ csu,   0],[ csu,   0],[   0,   0]], # link55、12→11   
    [[ csu,   0],[ csu,   0],[ csu,   0],[ csu,   0],[   0,   0]], # link56、13→3
    [[ csu,   0],[ csu,   0],[ csu,   0],[ csu,   0],[   0,   0]], # link57、13→4
    [[ csu,   0],[ csu,   0],[ csu,   0],[ csu,   0],[   0,   0]], # link58、13→5
    [[ csu,   0],[ csu,   0],[ csu,   0],[ csu,   0],[   0,   0]], # link59、13→6
    [[ csu,   0],[ csu,   0],[ csu,   0],[ csu,   0],[   0,   0]], # link60、13→7
    [[ csu,   0],[ csu,   0],[ csu,   0],[ csu,   0],[   0,   0]], # link61、13→8
    [[ csu,   0],[ csu,   0],[ csu,   0],[ csu,   0],[   0,   0]], # link62、13→9
    [[ csu,   0],[ csu,   0],[ csu,   0],[ csu,   0],[   0,   0]], # link63、13→10
    [[ csu,   0],[ csu,   0],[ csu,   0],[ csu,   0],[   0,   0]], # link64、13→11
    [[ csu,   0],[ csu,   0],[ csu,   0],[ csu,   0],[   0,   0]], # link65、14→3
    [[ csu,   0],[ csu,   0],[ csu,   0],[ csu,   0],[   0,   0]], # link66、14→4
    [[ csu,   0],[ csu,   0],[ csu,   0],[ csu,   0],[   0,   0]], # link67、14→5
    [[ csu,   0],[ csu,   0],[ csu,   0],[ csu,   0],[   0,   0]], # link68、14→6
    [[ csu,   0],[ csu,   0],[ csu,   0],[ csu,   0],[   0,   0]], # link69、14→7
    [[ csu,   0],[ csu,   0],[ csu,   0],[ csu,   0],[   0,   0]], # link70、14→8
    [[ csu,   0],[ csu,   0],[ csu,   0],[ csu,   0],[   0,   0]], # link71、14→9
    [[ csu,   0],[ csu,   0],[ csu,   0],[ csu,   0],[   0,   0]], # link72、14→10
    [[ csu,   0],[ csu,   0],[ csu,   0],[ csu,   0],[   0,   0]], # link73、14→11
    [[ csu,   0],[ csu,   0],[ csu,   0],[ csu,   0],[   0,   0]], # link74、15→3
    [[ csu,   0],[ csu,   0],[ csu,   0],[ csu,   0],[   0,   0]], # link75、15→4
    [[ csu,   0],[ csu,   0],[ csu,   0],[ csu,   0],[   0,   0]], # link76、15→5
    [[ csu,   0],[ csu,   0],[ csu,   0],[ csu,   0],[   0,   0]], # link77、15→6
    [[ csu,   0],[ csu,   0],[ csu,   0],[ csu,   0],[   0,   0]], # link78、15→7
    [[ csu,   0],[ csu,   0],[ csu,   0],[ csu,   0],[   0,   0]], # link79、15→8
    [[ csu,   0],[ csu,   0],[ csu,   0],[ csu,   0],[   0,   0]], # link80、15→9
    [[ csu,   0],[ csu,   0],[ csu,   0],[ csu,   0],[   0,   0]], # link81、15→10
    [[ csu,   0],[ csu,   0],[ csu,   0],[ csu,   0],[   0,   0]], # link82、15→11

#    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link83、16→3
#    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link84、16→8
#    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link85、17→4
#    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link86、17→9
#    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link87、18→5
#    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link88、18→10
#    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link89、19→6
#    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link90、19→11
# どちらか↑↓
    [[ csu,   0],[ csu,   0],[ csu,   0],[ csu,   0],[   0,   0]], # link83、16→3
    [[ csu,   0],[ csu,   0],[ csu,   0],[ csu,   0],[   0,   0]], # link84、16→8
    [[ csu,   0],[ csu,   0],[ csu,   0],[ csu,   0],[   0,   0]], # link85、17→4
    [[ csu,   0],[ csu,   0],[ csu,   0],[ csu,   0],[   0,   0]], # link86、17→9
    [[ csu,   0],[ csu,   0],[ csu,   0],[ csu,   0],[   0,   0]], # link87、18→5
    [[ csu,   0],[ csu,   0],[ csu,   0],[ csu,   0],[   0,   0]], # link88、18→10
    [[ csu,   0],[ csu,   0],[ csu,   0],[ csu,   0],[   0,   0]], # link89、19→6
    [[ csu,   0],[ csu,   0],[ csu,   0],[ csu,   0],[   0,   0]], # link90、19→11

    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link91、3→3　cbxはベースポートでの組立コスト
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link92、4→4
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link93、5→5
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link94、6→6

    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link95、8→8 cbsは設置海域での組立コスト
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link96、9→9
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link97、10→10
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link98、11→11
])

# 2年目以降
for m in range(1, N_network):
    c_s_use[m] = c_s_use[0].copy()

リンク建設コスト $cs_{start}$

In [173]:
# リンクを使用し始めるためにかかるコスト（建設費）
c_s_start = [m for m in range(N_network)]

# 1年目、建設可能性のある生産拠点から流出するリンクにかかる
c_s_start[0] = np.array([
    # link
    # outflow, inflow
    [  0,  0], [  0,  0], [  0,  0], [  0,  0], [  0,  0], [  0,  0], [  0,  0], [  0,  0], [  0,  0],
    [  0,  0], [  0,  0], [  0,  0], [  0,  0], [  0,  0], [  0,  0], [  0,  0], [  0,  0], [  0,  0],
    [  0,  0], [  0,  0], [  0,  0], [  0,  0], [  0,  0], [  0,  0], [  0,  0], [  0,  0], [  0,  0],
    [  0,  0], [  0,  0], [  0,  0], [  0,  0],
    [  0,  0], [  0,  0], [  0,  0], [  0,  0],
    [  0,  0], [  0,  0], [  0,  0], [  0,  0], 
    [  0,  0], [  0,  0], [  0,  0], [  0,  0],
    [  0,  0], [  0,  0], [  0,  0], [  0,  0],
    [css,  0], [css,  0], [css,  0], [css,  0], [css,  0], [css,  0], [css,  0], [css,  0], [css,  0], # link47 ~ link55
    [css,  0], [css,  0], [css,  0], [css,  0], [css,  0], [css,  0], [css,  0], [css,  0], [css,  0], # link56 ~ link64
    [css,  0], [css,  0], [css,  0], [css,  0], [css,  0], [css,  0], [css,  0], [css,  0], [css,  0], # link65 ~ link73
    [css,  0], [css,  0], [css,  0], [css,  0], [css,  0], [css,  0], [css,  0], [css,  0], [css,  0], # link74 ~ link82
    [  0,  0], [  0,  0], [  0,  0], [  0,  0], [  0,  0], [  0,  0], [  0,  0], [  0,  0],
    [  0,  0], [  0,  0], [  0,  0], [  0,  0], [  0,  0], [  0,  0], [  0,  0], [  0,  0], # グラフループ
])

# 2年目以降
for m in range(1, N_network):
    c_s_start[m] = c_s_start[0].copy()

キャパシティ $u$

In [174]:
u = [m for m in range(N_network)]

# 1年目
u[0] = np.array([
    # link
    # tower, blade, nacelle, foundation, product,
    # outflow, inflow
    [[   0,   0],[   0,   0],[  uy,  uy],[   0,   0],[   0,   0]], # link0、0→3
    [[   0,   0],[   0,   0],[  uy,  uy],[   0,   0],[   0,   0]], # link1、0→4
    [[   0,   0],[   0,   0],[  uy,  uy],[   0,   0],[   0,   0]], # link2、0→5
    [[   0,   0],[   0,   0],[  uy,  uy],[   0,   0],[   0,   0]], # link3、0→6
    [[   0,   0],[   0,   0],[  uy,  uy],[   0,   0],[   0,   0]], # link4、0→7
    [[   0,   0],[   0,   0],[  uy,  uy],[   0,   0],[   0,   0]], # link5、0→8
    [[   0,   0],[   0,   0],[  uy,  uy],[   0,   0],[   0,   0]], # link6、0→9
    [[   0,   0],[   0,   0],[  uy,  uy],[   0,   0],[   0,   0]], # link7、0→10
    [[   0,   0],[   0,   0],[  uy,  uy],[   0,   0],[   0,   0]], # link8、0→11
    [[   0,   0],[   0,   0],[   0,   0],[ uy_, uy_],[   0,   0]], # link9、1→3
    [[   0,   0],[   0,   0],[   0,   0],[ uy_, uy_],[   0,   0]], # link10、1→4
    [[   0,   0],[   0,   0],[   0,   0],[ uy_, uy_],[   0,   0]], # link11、1→5
    [[   0,   0],[   0,   0],[   0,   0],[ uy_, uy_],[   0,   0]], # link12、1→6
    [[   0,   0],[   0,   0],[   0,   0],[ uy_, uy_],[   0,   0]], # link13、1→7
    [[   0,   0],[   0,   0],[   0,   0],[ uy_, uy_],[   0,   0]], # link14、1→8
    [[   0,   0],[   0,   0],[   0,   0],[ uy_, uy_],[   0,   0]], # link15、1→9
    [[   0,   0],[   0,   0],[   0,   0],[ uy_, uy_],[   0,   0]], # link16、1→10
    [[   0,   0],[   0,   0],[   0,   0],[ uy_, uy_],[   0,   0]], # link17、1→11
    [[  uy,  uy],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link18、2→3
    [[  uy,  uy],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link19、2→4
    [[  uy,  uy],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link20、2→5
    [[  uy,  uy],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link21、2→6
    [[  uy,  uy],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link22、2→7
    [[  uy,  uy],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link23、2→8
    [[  uy,  uy],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link24、2→9
    [[  uy,  uy],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link25、2→10
    [[  uy,  uy],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link26、2→11

    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[  us,  us]], # link27、3→8
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[  us,  us]], # link28、3→9
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[  us,  us]], # link29、3→10
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[  us,  us]], # link30、3→11
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[  us,  us]], # link31、4→8
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[  us,  us]], # link32、4→9
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[  us,  us]], # link33、4→10
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[  us,  us]], # link34、4→11
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[  us,  us]], # link35、5→8
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[  us,  us]], # link36、5→9
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[  us,  us]], # link37、5→10
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[  us,  us]], # link38、5→11
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[  us,  us]], # link39、6→8
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[  us,  us]], # link40、6→9
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[  us,  us]], # link41、6→10
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[  us,  us]], # link42、6→11
    [[  us,  us],[3*us,3*us],[  us,  us],[  us,  us],[   0,   0]], # link43、7→8
    [[  us,  us],[3*us,3*us],[  us,  us],[  us,  us],[   0,   0]], # link44、7→9
    [[  us,  us],[3*us,3*us],[  us,  us],[  us,  us],[   0,   0]], # link45、7→10
    [[  us,  us],[3*us,3*us],[  us,  us],[  us,  us],[   0,   0]], # link46、7→11

    [[  uy,  uy],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link47、12→3
    [[  uy,  uy],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link48、12→4
    [[  uy,  uy],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link49、12→5
    [[  uy,  uy],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link50、12→6
    [[  uy,  uy],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link51、12→7
    [[  uy,  uy],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link52、12→8
    [[  uy,  uy],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link53、12→9
    [[  uy,  uy],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link54、12→10
    [[  uy,  uy],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link55、12→11   
    [[   0,   0],[   0,   0],[   0,   0],[ uy_, uy_],[   0,   0]], # link56、13→3
    [[   0,   0],[   0,   0],[   0,   0],[ uy_, uy_],[   0,   0]], # link57、13→4
    [[   0,   0],[   0,   0],[   0,   0],[ uy_, uy_],[   0,   0]], # link58、13→5
    [[   0,   0],[   0,   0],[   0,   0],[ uy_, uy_],[   0,   0]], # link59、13→6
    [[   0,   0],[   0,   0],[   0,   0],[ uy_, uy_],[   0,   0]], # link60、13→7
    [[   0,   0],[   0,   0],[   0,   0],[ uy_, uy_],[   0,   0]], # link61、13→8
    [[   0,   0],[   0,   0],[   0,   0],[ uy_, uy_],[   0,   0]], # link62、13→9
    [[   0,   0],[   0,   0],[   0,   0],[ uy_, uy_],[   0,   0]], # link63、13→10
    [[   0,   0],[   0,   0],[   0,   0],[ uy_, uy_],[   0,   0]], # link64、13→11
    [[   0,   0],[   0,   0],[  uy,  uy],[   0,   0],[   0,   0]], # link65、14→3
    [[   0,   0],[   0,   0],[  uy,  uy],[   0,   0],[   0,   0]], # link66、14→4
    [[   0,   0],[   0,   0],[  uy,  uy],[   0,   0],[   0,   0]], # link67、14→5
    [[   0,   0],[   0,   0],[  uy,  uy],[   0,   0],[   0,   0]], # link68、14→6
    [[   0,   0],[   0,   0],[  uy,  uy],[   0,   0],[   0,   0]], # link69、14→7
    [[   0,   0],[   0,   0],[  uy,  uy],[   0,   0],[   0,   0]], # link70、14→8
    [[   0,   0],[   0,   0],[  uy,  uy],[   0,   0],[   0,   0]], # link71、14→9
    [[   0,   0],[   0,   0],[  uy,  uy],[   0,   0],[   0,   0]], # link72、14→10
    [[   0,   0],[   0,   0],[  uy,  uy],[   0,   0],[   0,   0]], # link73、14→11
    [[   0,   0],[3*uy,3*uy],[   0,   0],[   0,   0],[   0,   0]], # link74、15→3
    [[   0,   0],[3*uy,3*uy],[   0,   0],[   0,   0],[   0,   0]], # link75、15→4
    [[   0,   0],[3*uy,3*uy],[   0,   0],[   0,   0],[   0,   0]], # link76、15→5
    [[   0,   0],[3*uy,3*uy],[   0,   0],[   0,   0],[   0,   0]], # link77、15→6
    [[   0,   0],[3*uy,3*uy],[   0,   0],[   0,   0],[   0,   0]], # link78、15→7
    [[   0,   0],[3*uy,3*uy],[   0,   0],[   0,   0],[   0,   0]], # link79、15→8
    [[   0,   0],[3*uy,3*uy],[   0,   0],[   0,   0],[   0,   0]], # link80、15→9
    [[   0,   0],[3*uy,3*uy],[   0,   0],[   0,   0],[   0,   0]], # link81、15→10
    [[   0,   0],[3*uy,3*uy],[   0,   0],[   0,   0],[   0,   0]], # link82、15→11

    [[   0,   0],[9999,9999],[   0,   0],[   0,   0],[   0,   0]], # link83、16→3
    [[   0,   0],[9999,9999],[   0,   0],[   0,   0],[   0,   0]], # link84、16→8
    [[   0,   0],[9999,9999],[   0,   0],[   0,   0],[   0,   0]], # link85、17→4
    [[   0,   0],[9999,9999],[   0,   0],[   0,   0],[   0,   0]], # link86、17→9
    [[   0,   0],[9999,9999],[   0,   0],[   0,   0],[   0,   0]], # link87、18→5
    [[   0,   0],[9999,9999],[   0,   0],[   0,   0],[   0,   0]], # link88、18→10
    [[   0,   0],[9999,9999],[   0,   0],[   0,   0],[   0,   0]], # link89、19→6
    [[   0,   0],[9999,9999],[   0,   0],[   0,   0],[   0,   0]], # link90、19→11

    [[  ub,   0],[3*ub,   0],[  ub,   0],[  ub,   0],[   0,  ub]], # link91、3→3
    [[  ub,   0],[3*ub,   0],[  ub,   0],[  ub,   0],[   0,  ub]], # link92、4→4
    [[  ub,   0],[3*ub,   0],[  ub,   0],[  ub,   0],[   0,  ub]], # link93、5→5
    [[  ub,   0],[3*ub,   0],[  ub,   0],[  ub,   0],[   0,  ub]], # link94、6→6
    [[ usa,   0],[3*usa,   0],[ usa,   0],[ usa,   0],[   0, usa]], # link95、8→8
    [[ usa,   0],[3*usa,   0],[ usa,   0],[ usa,   0],[   0, usa]], # link96、9→9
    [[ usa,   0],[3*usa,   0],[ usa,   0],[ usa,   0],[   0, usa]], # link97、10→10
    [[ usa,   0],[3*usa,   0],[ usa,   0],[ usa,   0],[   0, usa]], # link98、11→11
])

# 2年目
u[1] = np.array([
    # link
    # tower, blade, nacelle, foundation, product,
    # outflow, inflow
    [[   0,   0],[   0,   0],[  uy2,  uy2],[   0,   0],[   0,   0]], # link0、0→3
    [[   0,   0],[   0,   0],[  uy2,  uy2],[   0,   0],[   0,   0]], # link1、0→4
    [[   0,   0],[   0,   0],[  uy2,  uy2],[   0,   0],[   0,   0]], # link2、0→5
    [[   0,   0],[   0,   0],[  uy2,  uy2],[   0,   0],[   0,   0]], # link3、0→6
    [[   0,   0],[   0,   0],[  uy2,  uy2],[   0,   0],[   0,   0]], # link4、0→7
    [[   0,   0],[   0,   0],[  uy2,  uy2],[   0,   0],[   0,   0]], # link5、0→8
    [[   0,   0],[   0,   0],[  uy2,  uy2],[   0,   0],[   0,   0]], # link6、0→9
    [[   0,   0],[   0,   0],[  uy2,  uy2],[   0,   0],[   0,   0]], # link7、0→10
    [[   0,   0],[   0,   0],[  uy2,  uy2],[   0,   0],[   0,   0]], # link8、0→11
    [[   0,   0],[   0,   0],[   0,   0],[ uy2_, uy2_],[   0,   0]], # link9、1→3
    [[   0,   0],[   0,   0],[   0,   0],[ uy2_, uy2_],[   0,   0]], # link10、1→4
    [[   0,   0],[   0,   0],[   0,   0],[ uy2_, uy2_],[   0,   0]], # link11、1→5
    [[   0,   0],[   0,   0],[   0,   0],[ uy2_, uy2_],[   0,   0]], # link12、1→6
    [[   0,   0],[   0,   0],[   0,   0],[ uy2_, uy2_],[   0,   0]], # link13、1→7
    [[   0,   0],[   0,   0],[   0,   0],[ uy2_, uy2_],[   0,   0]], # link14、1→8
    [[   0,   0],[   0,   0],[   0,   0],[ uy2_, uy2_],[   0,   0]], # link15、1→9
    [[   0,   0],[   0,   0],[   0,   0],[ uy2_, uy2_],[   0,   0]], # link16、1→10
    [[   0,   0],[   0,   0],[   0,   0],[ uy2_, uy2_],[   0,   0]], # link17、1→11
    [[  uy2,  uy2],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link18、2→3
    [[  uy2,  uy2],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link19、2→4
    [[  uy2,  uy2],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link20、2→5
    [[  uy2,  uy2],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link21、2→6
    [[  uy2,  uy2],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link22、2→7
    [[  uy2,  uy2],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link23、2→8
    [[  uy2,  uy2],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link24、2→9
    [[  uy2,  uy2],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link25、2→10
    [[  uy2,  uy2],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link26、2→11

    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[  us2,  us2]], # link27、3→8
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[  us2,  us2]], # link28、3→9
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[  us2,  us2]], # link29、3→10
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[  us2,  us2]], # link30、3→11
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[  us2,  us2]], # link31、4→8
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[  us2,  us2]], # link32、4→9
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[  us2,  us2]], # link33、4→10
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[  us2,  us2]], # link34、4→11
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[  us2,  us2]], # link35、5→8
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[  us2,  us2]], # link36、5→9
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[  us2,  us2]], # link37、5→10
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[  us2,  us2]], # link38、5→11
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[  us2,  us2]], # link39、6→8
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[  us2,  us2]], # link40、6→9
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[  us2,  us2]], # link41、6→10
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[  us2,  us2]], # link42、6→11
    [[  us2,  us2],[3*us2,3*us2],[  us2,  us2],[  us2,  us2],[   0,   0]], # link43、7→8
    [[  us2,  us2],[3*us2,3*us2],[  us2,  us2],[  us2,  us2],[   0,   0]], # link44、7→9
    [[  us2,  us2],[3*us2,3*us2],[  us2,  us2],[  us2,  us2],[   0,   0]], # link45、7→10
    [[  us2,  us2],[3*us2,3*us2],[  us2,  us2],[  us2,  us2],[   0,   0]], # link46、7→11

    [[  uy2,  uy2],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link47、12→3
    [[  uy2,  uy2],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link48、12→4
    [[  uy2,  uy2],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link49、12→5
    [[  uy2,  uy2],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link50、12→6
    [[  uy2,  uy2],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link51、12→7
    [[  uy2,  uy2],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link52、12→8
    [[  uy2,  uy2],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link53、12→9
    [[  uy2,  uy2],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link54、12→10
    [[  uy2,  uy2],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link55、12→11   
    [[   0,   0],[   0,   0],[   0,   0],[ uy2_, uy2_],[   0,   0]], # link56、13→3
    [[   0,   0],[   0,   0],[   0,   0],[ uy2_, uy2_],[   0,   0]], # link57、13→4
    [[   0,   0],[   0,   0],[   0,   0],[ uy2_, uy2_],[   0,   0]], # link58、13→5
    [[   0,   0],[   0,   0],[   0,   0],[ uy2_, uy2_],[   0,   0]], # link59、13→6
    [[   0,   0],[   0,   0],[   0,   0],[ uy2_, uy2_],[   0,   0]], # link60、13→7
    [[   0,   0],[   0,   0],[   0,   0],[ uy2_, uy2_],[   0,   0]], # link61、13→8
    [[   0,   0],[   0,   0],[   0,   0],[ uy2_, uy2_],[   0,   0]], # link62、13→9
    [[   0,   0],[   0,   0],[   0,   0],[ uy2_, uy2_],[   0,   0]], # link63、13→10
    [[   0,   0],[   0,   0],[   0,   0],[ uy2_, uy2_],[   0,   0]], # link64、13→11
    [[   0,   0],[   0,   0],[  uy2,  uy2],[   0,   0],[   0,   0]], # link65、14→3
    [[   0,   0],[   0,   0],[  uy2,  uy2],[   0,   0],[   0,   0]], # link66、14→4
    [[   0,   0],[   0,   0],[  uy2,  uy2],[   0,   0],[   0,   0]], # link67、14→5
    [[   0,   0],[   0,   0],[  uy2,  uy2],[   0,   0],[   0,   0]], # link68、14→6
    [[   0,   0],[   0,   0],[  uy2,  uy2],[   0,   0],[   0,   0]], # link69、14→7
    [[   0,   0],[   0,   0],[  uy2,  uy2],[   0,   0],[   0,   0]], # link70、14→8
    [[   0,   0],[   0,   0],[  uy2,  uy2],[   0,   0],[   0,   0]], # link71、14→9
    [[   0,   0],[   0,   0],[  uy2,  uy2],[   0,   0],[   0,   0]], # link72、14→10
    [[   0,   0],[   0,   0],[  uy2,  uy2],[   0,   0],[   0,   0]], # link73、14→11
    [[   0,   0],[3*uy2,3*uy2],[   0,   0],[   0,   0],[   0,   0]], # link74、15→3
    [[   0,   0],[3*uy2,3*uy2],[   0,   0],[   0,   0],[   0,   0]], # link75、15→4
    [[   0,   0],[3*uy2,3*uy2],[   0,   0],[   0,   0],[   0,   0]], # link76、15→5
    [[   0,   0],[3*uy2,3*uy2],[   0,   0],[   0,   0],[   0,   0]], # link77、15→6
    [[   0,   0],[3*uy2,3*uy2],[   0,   0],[   0,   0],[   0,   0]], # link78、15→7
    [[   0,   0],[3*uy2,3*uy2],[   0,   0],[   0,   0],[   0,   0]], # link79、15→8
    [[   0,   0],[3*uy2,3*uy2],[   0,   0],[   0,   0],[   0,   0]], # link80、15→9
    [[   0,   0],[3*uy2,3*uy2],[   0,   0],[   0,   0],[   0,   0]], # link81、15→10
    [[   0,   0],[3*uy2,3*uy2],[   0,   0],[   0,   0],[   0,   0]], # link82、15→11

    [[   0,   0],[9999,9999],[   0,   0],[   0,   0],[   0,   0]], # link83、16→3
    [[   0,   0],[9999,9999],[   0,   0],[   0,   0],[   0,   0]], # link84、16→8
    [[   0,   0],[9999,9999],[   0,   0],[   0,   0],[   0,   0]], # link85、17→4
    [[   0,   0],[9999,9999],[   0,   0],[   0,   0],[   0,   0]], # link86、17→9
    [[   0,   0],[9999,9999],[   0,   0],[   0,   0],[   0,   0]], # link87、18→5
    [[   0,   0],[9999,9999],[   0,   0],[   0,   0],[   0,   0]], # link88、18→10
    [[   0,   0],[9999,9999],[   0,   0],[   0,   0],[   0,   0]], # link89、19→6
    [[   0,   0],[9999,9999],[   0,   0],[   0,   0],[   0,   0]], # link90、19→11

    [[  ub2,   0],[3*ub2,   0],[  ub2,   0],[  ub2,   0],[   0,  ub2]], # link91、3→3
    [[  ub2,   0],[3*ub2,   0],[  ub2,   0],[  ub2,   0],[   0,  ub2]], # link92、4→4
    [[  ub2,   0],[3*ub2,   0],[  ub2,   0],[  ub2,   0],[   0,  ub2]], # link93、5→5
    [[  ub2,   0],[3*ub2,   0],[  ub2,   0],[  ub2,   0],[   0,  ub2]], # link94、6→6
    [[ usa2,   0],[3*usa2,   0],[ usa2,   0],[ usa2,   0],[   0, usa2]], # link95、8→8
    [[ usa2,   0],[3*usa2,   0],[ usa2,   0],[ usa2,   0],[   0, usa2]], # link96、9→9
    [[ usa2,   0],[3*usa2,   0],[ usa2,   0],[ usa2,   0],[   0, usa2]], # link97、10→10
    [[ usa2,   0],[3*usa2,   0],[ usa2,   0],[ usa2,   0],[   0, usa2]], # link98、11→11
])

# 3年目
u[2] = np.array([
    # link
    # tower, blade, nacelle, foundation, product,
    # outflow, inflow
    [[   0,   0],[   0,   0],[  uy3,  uy3],[   0,   0],[   0,   0]], # link0、0→3
    [[   0,   0],[   0,   0],[  uy3,  uy3],[   0,   0],[   0,   0]], # link1、0→4
    [[   0,   0],[   0,   0],[  uy3,  uy3],[   0,   0],[   0,   0]], # link2、0→5
    [[   0,   0],[   0,   0],[  uy3,  uy3],[   0,   0],[   0,   0]], # link3、0→6
    [[   0,   0],[   0,   0],[  uy3,  uy3],[   0,   0],[   0,   0]], # link4、0→7
    [[   0,   0],[   0,   0],[  uy3,  uy3],[   0,   0],[   0,   0]], # link5、0→8
    [[   0,   0],[   0,   0],[  uy3,  uy3],[   0,   0],[   0,   0]], # link6、0→9
    [[   0,   0],[   0,   0],[  uy3,  uy3],[   0,   0],[   0,   0]], # link7、0→10
    [[   0,   0],[   0,   0],[  uy3,  uy3],[   0,   0],[   0,   0]], # link8、0→11
    [[   0,   0],[   0,   0],[   0,   0],[ uy3_, uy3_],[   0,   0]], # link9、1→3
    [[   0,   0],[   0,   0],[   0,   0],[ uy3_, uy3_],[   0,   0]], # link10、1→4
    [[   0,   0],[   0,   0],[   0,   0],[ uy3_, uy3_],[   0,   0]], # link11、1→5
    [[   0,   0],[   0,   0],[   0,   0],[ uy3_, uy3_],[   0,   0]], # link12、1→6
    [[   0,   0],[   0,   0],[   0,   0],[ uy3_, uy3_],[   0,   0]], # link13、1→7
    [[   0,   0],[   0,   0],[   0,   0],[ uy3_, uy3_],[   0,   0]], # link14、1→8
    [[   0,   0],[   0,   0],[   0,   0],[ uy3_, uy3_],[   0,   0]], # link15、1→9
    [[   0,   0],[   0,   0],[   0,   0],[ uy3_, uy3_],[   0,   0]], # link16、1→10
    [[   0,   0],[   0,   0],[   0,   0],[ uy3_, uy3_],[   0,   0]], # link17、1→11
    [[  uy3,  uy3],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link18、2→3
    [[  uy3,  uy3],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link19、2→4
    [[  uy3,  uy3],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link20、2→5
    [[  uy3,  uy3],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link21、2→6
    [[  uy3,  uy3],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link22、2→7
    [[  uy3,  uy3],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link23、2→8
    [[  uy3,  uy3],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link24、2→9
    [[  uy3,  uy3],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link25、2→10
    [[  uy3,  uy3],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link26、2→11

    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[  us3,  us3]], # link27、3→8
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[  us3,  us3]], # link28、3→9
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[  us3,  us3]], # link29、3→10
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[  us3,  us3]], # link30、3→11
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[  us3,  us3]], # link31、4→8
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[  us3,  us3]], # link32、4→9
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[  us3,  us3]], # link33、4→10
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[  us3,  us3]], # link34、4→11
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[  us3,  us3]], # link35、5→8
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[  us3,  us3]], # link36、5→9
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[  us3,  us3]], # link37、5→10
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[  us3,  us3]], # link38、5→11
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[  us3,  us3]], # link39、6→8
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[  us3,  us3]], # link40、6→9
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[  us3,  us3]], # link41、6→10
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[  us3,  us3]], # link42、6→11
    [[  us3,  us3],[3*us3,3*us3],[  us3,  us3],[  us3,  us3],[   0,   0]], # link43、7→8
    [[  us3,  us3],[3*us3,3*us3],[  us3,  us3],[  us3,  us3],[   0,   0]], # link44、7→9
    [[  us3,  us3],[3*us3,3*us3],[  us3,  us3],[  us3,  us3],[   0,   0]], # link45、7→10
    [[  us3,  us3],[3*us3,3*us3],[  us3,  us3],[  us3,  us3],[   0,   0]], # link46、7→11

    [[  uy3,  uy3],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link47、12→3
    [[  uy3,  uy3],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link48、12→4
    [[  uy3,  uy3],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link49、12→5
    [[  uy3,  uy3],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link50、12→6
    [[  uy3,  uy3],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link51、12→7
    [[  uy3,  uy3],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link52、12→8
    [[  uy3,  uy3],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link53、12→9
    [[  uy3,  uy3],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link54、12→10
    [[  uy3,  uy3],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link55、12→11   
    [[   0,   0],[   0,   0],[   0,   0],[ uy3_, uy3_],[   0,   0]], # link56、13→3
    [[   0,   0],[   0,   0],[   0,   0],[ uy3_, uy3_],[   0,   0]], # link57、13→4
    [[   0,   0],[   0,   0],[   0,   0],[ uy3_, uy3_],[   0,   0]], # link58、13→5
    [[   0,   0],[   0,   0],[   0,   0],[ uy3_, uy3_],[   0,   0]], # link59、13→6
    [[   0,   0],[   0,   0],[   0,   0],[ uy3_, uy3_],[   0,   0]], # link60、13→7
    [[   0,   0],[   0,   0],[   0,   0],[ uy3_, uy3_],[   0,   0]], # link61、13→8
    [[   0,   0],[   0,   0],[   0,   0],[ uy3_, uy3_],[   0,   0]], # link62、13→9
    [[   0,   0],[   0,   0],[   0,   0],[ uy3_, uy3_],[   0,   0]], # link63、13→10
    [[   0,   0],[   0,   0],[   0,   0],[ uy3_, uy3_],[   0,   0]], # link64、13→11
    [[   0,   0],[   0,   0],[  uy3,  uy3],[   0,   0],[   0,   0]], # link65、14→3
    [[   0,   0],[   0,   0],[  uy3,  uy3],[   0,   0],[   0,   0]], # link66、14→4
    [[   0,   0],[   0,   0],[  uy3,  uy3],[   0,   0],[   0,   0]], # link67、14→5
    [[   0,   0],[   0,   0],[  uy3,  uy3],[   0,   0],[   0,   0]], # link68、14→6
    [[   0,   0],[   0,   0],[  uy3,  uy3],[   0,   0],[   0,   0]], # link69、14→7
    [[   0,   0],[   0,   0],[  uy3,  uy3],[   0,   0],[   0,   0]], # link70、14→8
    [[   0,   0],[   0,   0],[  uy3,  uy3],[   0,   0],[   0,   0]], # link71、14→9
    [[   0,   0],[   0,   0],[  uy3,  uy3],[   0,   0],[   0,   0]], # link72、14→10
    [[   0,   0],[   0,   0],[  uy3,  uy3],[   0,   0],[   0,   0]], # link73、14→11
    [[   0,   0],[3*uy3,3*uy3],[   0,   0],[   0,   0],[   0,   0]], # link74、15→3
    [[   0,   0],[3*uy3,3*uy3],[   0,   0],[   0,   0],[   0,   0]], # link75、15→4
    [[   0,   0],[3*uy3,3*uy3],[   0,   0],[   0,   0],[   0,   0]], # link76、15→5
    [[   0,   0],[3*uy3,3*uy3],[   0,   0],[   0,   0],[   0,   0]], # link77、15→6
    [[   0,   0],[3*uy3,3*uy3],[   0,   0],[   0,   0],[   0,   0]], # link78、15→7
    [[   0,   0],[3*uy3,3*uy3],[   0,   0],[   0,   0],[   0,   0]], # link79、15→8
    [[   0,   0],[3*uy3,3*uy3],[   0,   0],[   0,   0],[   0,   0]], # link80、15→9
    [[   0,   0],[3*uy3,3*uy3],[   0,   0],[   0,   0],[   0,   0]], # link81、15→10
    [[   0,   0],[3*uy3,3*uy3],[   0,   0],[   0,   0],[   0,   0]], # link82、15→11

    [[   0,   0],[9999,9999],[   0,   0],[   0,   0],[   0,   0]], # link83、16→3
    [[   0,   0],[9999,9999],[   0,   0],[   0,   0],[   0,   0]], # link84、16→8
    [[   0,   0],[9999,9999],[   0,   0],[   0,   0],[   0,   0]], # link85、17→4
    [[   0,   0],[9999,9999],[   0,   0],[   0,   0],[   0,   0]], # link86、17→9
    [[   0,   0],[9999,9999],[   0,   0],[   0,   0],[   0,   0]], # link87、18→5
    [[   0,   0],[9999,9999],[   0,   0],[   0,   0],[   0,   0]], # link88、18→10
    [[   0,   0],[9999,9999],[   0,   0],[   0,   0],[   0,   0]], # link89、19→6
    [[   0,   0],[9999,9999],[   0,   0],[   0,   0],[   0,   0]], # link90、19→11

    [[  ub3,   0],[3*ub3,   0],[  ub3,   0],[  ub3,   0],[   0,  ub3]], # link91、3→3
    [[  ub3,   0],[3*ub3,   0],[  ub3,   0],[  ub3,   0],[   0,  ub3]], # link92、4→4
    [[  ub3,   0],[3*ub3,   0],[  ub3,   0],[  ub3,   0],[   0,  ub3]], # link93、5→5
    [[  ub3,   0],[3*ub3,   0],[  ub3,   0],[  ub3,   0],[   0,  ub3]], # link94、6→6
    [[ usa3,   0],[3*usa3,   0],[ usa3,   0],[ usa3,   0],[   0, usa3]], # link95、8→8
    [[ usa3,   0],[3*usa3,   0],[ usa3,   0],[ usa3,   0],[   0, usa3]], # link96、9→9
    [[ usa3,   0],[3*usa3,   0],[ usa3,   0],[ usa3,   0],[   0, usa3]], # link97、10→10
    [[ usa3,   0],[3*usa3,   0],[ usa3,   0],[ usa3,   0],[   0, usa3]], # link98、11→11
])

# 4年目
u[3] = np.array([
    # link
    # tower, blade, nacelle, foundation, product,
    # outflow, inflow
    [[   0,   0],[   0,   0],[  uy4,  uy4],[   0,   0],[   0,   0]], # link0、0→3
    [[   0,   0],[   0,   0],[  uy4,  uy4],[   0,   0],[   0,   0]], # link1、0→4
    [[   0,   0],[   0,   0],[  uy4,  uy4],[   0,   0],[   0,   0]], # link2、0→5
    [[   0,   0],[   0,   0],[  uy4,  uy4],[   0,   0],[   0,   0]], # link3、0→6
    [[   0,   0],[   0,   0],[  uy4,  uy4],[   0,   0],[   0,   0]], # link4、0→7
    [[   0,   0],[   0,   0],[  uy4,  uy4],[   0,   0],[   0,   0]], # link5、0→8
    [[   0,   0],[   0,   0],[  uy4,  uy4],[   0,   0],[   0,   0]], # link6、0→9
    [[   0,   0],[   0,   0],[  uy4,  uy4],[   0,   0],[   0,   0]], # link7、0→10
    [[   0,   0],[   0,   0],[  uy4,  uy4],[   0,   0],[   0,   0]], # link8、0→11
    [[   0,   0],[   0,   0],[   0,   0],[ uy4_, uy4_],[   0,   0]], # link9、1→3
    [[   0,   0],[   0,   0],[   0,   0],[ uy4_, uy4_],[   0,   0]], # link10、1→4
    [[   0,   0],[   0,   0],[   0,   0],[ uy4_, uy4_],[   0,   0]], # link11、1→5
    [[   0,   0],[   0,   0],[   0,   0],[ uy4_, uy4_],[   0,   0]], # link12、1→6
    [[   0,   0],[   0,   0],[   0,   0],[ uy4_, uy4_],[   0,   0]], # link13、1→7
    [[   0,   0],[   0,   0],[   0,   0],[ uy4_, uy4_],[   0,   0]], # link14、1→8
    [[   0,   0],[   0,   0],[   0,   0],[ uy4_, uy4_],[   0,   0]], # link15、1→9
    [[   0,   0],[   0,   0],[   0,   0],[ uy4_, uy4_],[   0,   0]], # link16、1→10
    [[   0,   0],[   0,   0],[   0,   0],[ uy4_, uy4_],[   0,   0]], # link17、1→11
    [[  uy4,  uy4],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link18、2→3
    [[  uy4,  uy4],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link19、2→4
    [[  uy4,  uy4],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link20、2→5
    [[  uy4,  uy4],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link21、2→6
    [[  uy4,  uy4],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link22、2→7
    [[  uy4,  uy4],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link23、2→8
    [[  uy4,  uy4],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link24、2→9
    [[  uy4,  uy4],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link25、2→10
    [[  uy4,  uy4],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link26、2→11

    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[  us4,  us4]], # link27、3→8
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[  us4,  us4]], # link28、3→9
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[  us4,  us4]], # link29、3→10
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[  us4,  us4]], # link30、3→11
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[  us4,  us4]], # link31、4→8
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[  us4,  us4]], # link32、4→9
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[  us4,  us4]], # link33、4→10
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[  us4,  us4]], # link34、4→11
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[  us4,  us4]], # link35、5→8
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[  us4,  us4]], # link36、5→9
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[  us4,  us4]], # link37、5→10
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[  us4,  us4]], # link38、5→11
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[  us4,  us4]], # link39、6→8
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[  us4,  us4]], # link40、6→9
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[  us4,  us4]], # link41、6→10
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[  us4,  us4]], # link42、6→11
    [[  us4,  us4],[3*us4,3*us4],[  us4,  us4],[  us4,  us4],[   0,   0]], # link43、7→8
    [[  us4,  us4],[3*us4,3*us4],[  us4,  us4],[  us4,  us4],[   0,   0]], # link44、7→9
    [[  us4,  us4],[3*us4,3*us4],[  us4,  us4],[  us4,  us4],[   0,   0]], # link45、7→10
    [[  us4,  us4],[3*us4,3*us4],[  us4,  us4],[  us4,  us4],[   0,   0]], # link46、7→11

    [[  uy4,  uy4],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link47、12→3
    [[  uy4,  uy4],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link48、12→4
    [[  uy4,  uy4],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link49、12→5
    [[  uy4,  uy4],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link50、12→6
    [[  uy4,  uy4],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link51、12→7
    [[  uy4,  uy4],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link52、12→8
    [[  uy4,  uy4],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link53、12→9
    [[  uy4,  uy4],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link54、12→10
    [[  uy4,  uy4],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link55、12→11   
    [[   0,   0],[   0,   0],[   0,   0],[ uy4_, uy4_],[   0,   0]], # link56、13→3
    [[   0,   0],[   0,   0],[   0,   0],[ uy4_, uy4_],[   0,   0]], # link57、13→4
    [[   0,   0],[   0,   0],[   0,   0],[ uy4_, uy4_],[   0,   0]], # link58、13→5
    [[   0,   0],[   0,   0],[   0,   0],[ uy4_, uy4_],[   0,   0]], # link59、13→6
    [[   0,   0],[   0,   0],[   0,   0],[ uy4_, uy4_],[   0,   0]], # link60、13→7
    [[   0,   0],[   0,   0],[   0,   0],[ uy4_, uy4_],[   0,   0]], # link61、13→8
    [[   0,   0],[   0,   0],[   0,   0],[ uy4_, uy4_],[   0,   0]], # link62、13→9
    [[   0,   0],[   0,   0],[   0,   0],[ uy4_, uy4_],[   0,   0]], # link63、13→10
    [[   0,   0],[   0,   0],[   0,   0],[ uy4_, uy4_],[   0,   0]], # link64、13→11
    [[   0,   0],[   0,   0],[  uy4,  uy4],[   0,   0],[   0,   0]], # link65、14→3
    [[   0,   0],[   0,   0],[  uy4,  uy4],[   0,   0],[   0,   0]], # link66、14→4
    [[   0,   0],[   0,   0],[  uy4,  uy4],[   0,   0],[   0,   0]], # link67、14→5
    [[   0,   0],[   0,   0],[  uy4,  uy4],[   0,   0],[   0,   0]], # link68、14→6
    [[   0,   0],[   0,   0],[  uy4,  uy4],[   0,   0],[   0,   0]], # link69、14→7
    [[   0,   0],[   0,   0],[  uy4,  uy4],[   0,   0],[   0,   0]], # link70、14→8
    [[   0,   0],[   0,   0],[  uy4,  uy4],[   0,   0],[   0,   0]], # link71、14→9
    [[   0,   0],[   0,   0],[  uy4,  uy4],[   0,   0],[   0,   0]], # link72、14→10
    [[   0,   0],[   0,   0],[  uy4,  uy4],[   0,   0],[   0,   0]], # link73、14→11
    [[   0,   0],[3*uy4,3*uy4],[   0,   0],[   0,   0],[   0,   0]], # link74、15→3
    [[   0,   0],[3*uy4,3*uy4],[   0,   0],[   0,   0],[   0,   0]], # link75、15→4
    [[   0,   0],[3*uy4,3*uy4],[   0,   0],[   0,   0],[   0,   0]], # link76、15→5
    [[   0,   0],[3*uy4,3*uy4],[   0,   0],[   0,   0],[   0,   0]], # link77、15→6
    [[   0,   0],[3*uy4,3*uy4],[   0,   0],[   0,   0],[   0,   0]], # link78、15→7
    [[   0,   0],[3*uy4,3*uy4],[   0,   0],[   0,   0],[   0,   0]], # link79、15→8
    [[   0,   0],[3*uy4,3*uy4],[   0,   0],[   0,   0],[   0,   0]], # link80、15→9
    [[   0,   0],[3*uy4,3*uy4],[   0,   0],[   0,   0],[   0,   0]], # link81、15→10
    [[   0,   0],[3*uy4,3*uy4],[   0,   0],[   0,   0],[   0,   0]], # link82、15→11

    [[   0,   0],[9999,9999],[   0,   0],[   0,   0],[   0,   0]], # link83、16→3
    [[   0,   0],[9999,9999],[   0,   0],[   0,   0],[   0,   0]], # link84、16→8
    [[   0,   0],[9999,9999],[   0,   0],[   0,   0],[   0,   0]], # link85、17→4
    [[   0,   0],[9999,9999],[   0,   0],[   0,   0],[   0,   0]], # link86、17→9
    [[   0,   0],[9999,9999],[   0,   0],[   0,   0],[   0,   0]], # link87、18→5
    [[   0,   0],[9999,9999],[   0,   0],[   0,   0],[   0,   0]], # link88、18→10
    [[   0,   0],[9999,9999],[   0,   0],[   0,   0],[   0,   0]], # link89、19→6
    [[   0,   0],[9999,9999],[   0,   0],[   0,   0],[   0,   0]], # link90、19→11

    [[  ub4,   0],[3*ub4,   0],[  ub4,   0],[  ub4,   0],[   0,  ub4]], # link91、3→3
    [[  ub4,   0],[3*ub4,   0],[  ub4,   0],[  ub4,   0],[   0,  ub4]], # link92、4→4
    [[  ub4,   0],[3*ub4,   0],[  ub4,   0],[  ub4,   0],[   0,  ub4]], # link93、5→5
    [[  ub4,   0],[3*ub4,   0],[  ub4,   0],[  ub4,   0],[   0,  ub4]], # link94、6→6
    [[ usa4,   0],[3*usa4,   0],[ usa4,   0],[ usa4,   0],[   0, usa4]], # link95、8→8
    [[ usa4,   0],[3*usa4,   0],[ usa4,   0],[ usa4,   0],[   0, usa4]], # link96、9→9
    [[ usa4,   0],[3*usa4,   0],[ usa4,   0],[ usa4,   0],[   0, usa4]], # link97、10→10
    [[ usa4,   0],[3*usa4,   0],[ usa4,   0],[ usa4,   0],[   0, usa4]], # link98、11→11
])

# 5年目
u[4] = np.array([
    # link
    # tower, blade, nacelle, foundation, product,
    # outflow, inflow
    [[   0,   0],[   0,   0],[  uy5,  uy5],[   0,   0],[   0,   0]], # link0、0→3
    [[   0,   0],[   0,   0],[  uy5,  uy5],[   0,   0],[   0,   0]], # link1、0→4
    [[   0,   0],[   0,   0],[  uy5,  uy5],[   0,   0],[   0,   0]], # link2、0→5
    [[   0,   0],[   0,   0],[  uy5,  uy5],[   0,   0],[   0,   0]], # link3、0→6
    [[   0,   0],[   0,   0],[  uy5,  uy5],[   0,   0],[   0,   0]], # link4、0→7
    [[   0,   0],[   0,   0],[  uy5,  uy5],[   0,   0],[   0,   0]], # link5、0→8
    [[   0,   0],[   0,   0],[  uy5,  uy5],[   0,   0],[   0,   0]], # link6、0→9
    [[   0,   0],[   0,   0],[  uy5,  uy5],[   0,   0],[   0,   0]], # link7、0→10
    [[   0,   0],[   0,   0],[  uy5,  uy5],[   0,   0],[   0,   0]], # link8、0→11
    [[   0,   0],[   0,   0],[   0,   0],[ uy5_, uy5_],[   0,   0]], # link9、1→3
    [[   0,   0],[   0,   0],[   0,   0],[ uy5_, uy5_],[   0,   0]], # link10、1→4
    [[   0,   0],[   0,   0],[   0,   0],[ uy5_, uy5_],[   0,   0]], # link11、1→5
    [[   0,   0],[   0,   0],[   0,   0],[ uy5_, uy5_],[   0,   0]], # link12、1→6
    [[   0,   0],[   0,   0],[   0,   0],[ uy5_, uy5_],[   0,   0]], # link13、1→7
    [[   0,   0],[   0,   0],[   0,   0],[ uy5_, uy5_],[   0,   0]], # link14、1→8
    [[   0,   0],[   0,   0],[   0,   0],[ uy5_, uy5_],[   0,   0]], # link15、1→9
    [[   0,   0],[   0,   0],[   0,   0],[ uy5_, uy5_],[   0,   0]], # link16、1→10
    [[   0,   0],[   0,   0],[   0,   0],[ uy5_, uy5_],[   0,   0]], # link17、1→11
    [[  uy5,  uy5],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link18、2→3
    [[  uy5,  uy5],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link19、2→4
    [[  uy5,  uy5],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link20、2→5
    [[  uy5,  uy5],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link21、2→6
    [[  uy5,  uy5],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link22、2→7
    [[  uy5,  uy5],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link23、2→8
    [[  uy5,  uy5],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link24、2→9
    [[  uy5,  uy5],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link25、2→10
    [[  uy5,  uy5],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link26、2→11

    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[  us5,  us5]], # link27、3→8
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[  us5,  us5]], # link28、3→9
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[  us5,  us5]], # link29、3→10
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[  us5,  us5]], # link30、3→11
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[  us5,  us5]], # link31、4→8
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[  us5,  us5]], # link32、4→9
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[  us5,  us5]], # link33、4→10
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[  us5,  us5]], # link34、4→11
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[  us5,  us5]], # link35、5→8
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[  us5,  us5]], # link36、5→9
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[  us5,  us5]], # link37、5→10
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[  us5,  us5]], # link38、5→11
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[  us5,  us5]], # link39、6→8
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[  us5,  us5]], # link40、6→9
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[  us5,  us5]], # link41、6→10
    [[   0,   0],[   0,   0],[   0,   0],[   0,   0],[  us5,  us5]], # link42、6→11
    [[  us5,  us5],[3*us5,3*us5],[  us5,  us5],[  us5,  us5],[   0,   0]], # link43、7→8
    [[  us5,  us5],[3*us5,3*us5],[  us5,  us5],[  us5,  us5],[   0,   0]], # link44、7→9
    [[  us5,  us5],[3*us5,3*us5],[  us5,  us5],[  us5,  us5],[   0,   0]], # link45、7→10
    [[  us5,  us5],[3*us5,3*us5],[  us5,  us5],[  us5,  us5],[   0,   0]], # link46、7→11

    [[  uy5,  uy5],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link47、12→3
    [[  uy5,  uy5],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link48、12→4
    [[  uy5,  uy5],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link49、12→5
    [[  uy5,  uy5],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link50、12→6
    [[  uy5,  uy5],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link51、12→7
    [[  uy5,  uy5],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link52、12→8
    [[  uy5,  uy5],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link53、12→9
    [[  uy5,  uy5],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link54、12→10
    [[  uy5,  uy5],[   0,   0],[   0,   0],[   0,   0],[   0,   0]], # link55、12→11   
    [[   0,   0],[   0,   0],[   0,   0],[ uy5_, uy5_],[   0,   0]], # link56、13→3
    [[   0,   0],[   0,   0],[   0,   0],[ uy5_, uy5_],[   0,   0]], # link57、13→4
    [[   0,   0],[   0,   0],[   0,   0],[ uy5_, uy5_],[   0,   0]], # link58、13→5
    [[   0,   0],[   0,   0],[   0,   0],[ uy5_, uy5_],[   0,   0]], # link59、13→6
    [[   0,   0],[   0,   0],[   0,   0],[ uy5_, uy5_],[   0,   0]], # link60、13→7
    [[   0,   0],[   0,   0],[   0,   0],[ uy5_, uy5_],[   0,   0]], # link61、13→8
    [[   0,   0],[   0,   0],[   0,   0],[ uy5_, uy5_],[   0,   0]], # link62、13→9
    [[   0,   0],[   0,   0],[   0,   0],[ uy5_, uy5_],[   0,   0]], # link63、13→10
    [[   0,   0],[   0,   0],[   0,   0],[ uy5_, uy5_],[   0,   0]], # link64、13→11
    [[   0,   0],[   0,   0],[  uy5,  uy5],[   0,   0],[   0,   0]], # link65、14→3
    [[   0,   0],[   0,   0],[  uy5,  uy5],[   0,   0],[   0,   0]], # link66、14→4
    [[   0,   0],[   0,   0],[  uy5,  uy5],[   0,   0],[   0,   0]], # link67、14→5
    [[   0,   0],[   0,   0],[  uy5,  uy5],[   0,   0],[   0,   0]], # link68、14→6
    [[   0,   0],[   0,   0],[  uy5,  uy5],[   0,   0],[   0,   0]], # link69、14→7
    [[   0,   0],[   0,   0],[  uy5,  uy5],[   0,   0],[   0,   0]], # link70、14→8
    [[   0,   0],[   0,   0],[  uy5,  uy5],[   0,   0],[   0,   0]], # link71、14→9
    [[   0,   0],[   0,   0],[  uy5,  uy5],[   0,   0],[   0,   0]], # link72、14→10
    [[   0,   0],[   0,   0],[  uy5,  uy5],[   0,   0],[   0,   0]], # link73、14→11
    [[   0,   0],[3*uy5,3*uy5],[   0,   0],[   0,   0],[   0,   0]], # link74、15→3
    [[   0,   0],[3*uy5,3*uy5],[   0,   0],[   0,   0],[   0,   0]], # link75、15→4
    [[   0,   0],[3*uy5,3*uy5],[   0,   0],[   0,   0],[   0,   0]], # link76、15→5
    [[   0,   0],[3*uy5,3*uy5],[   0,   0],[   0,   0],[   0,   0]], # link77、15→6
    [[   0,   0],[3*uy5,3*uy5],[   0,   0],[   0,   0],[   0,   0]], # link78、15→7
    [[   0,   0],[3*uy5,3*uy5],[   0,   0],[   0,   0],[   0,   0]], # link79、15→8
    [[   0,   0],[3*uy5,3*uy5],[   0,   0],[   0,   0],[   0,   0]], # link80、15→9
    [[   0,   0],[3*uy5,3*uy5],[   0,   0],[   0,   0],[   0,   0]], # link81、15→10
    [[   0,   0],[3*uy5,3*uy5],[   0,   0],[   0,   0],[   0,   0]], # link82、15→11

    [[   0,   0],[9999,9999],[   0,   0],[   0,   0],[   0,   0]], # link83、16→3
    [[   0,   0],[9999,9999],[   0,   0],[   0,   0],[   0,   0]], # link84、16→8
    [[   0,   0],[9999,9999],[   0,   0],[   0,   0],[   0,   0]], # link85、17→4
    [[   0,   0],[9999,9999],[   0,   0],[   0,   0],[   0,   0]], # link86、17→9
    [[   0,   0],[9999,9999],[   0,   0],[   0,   0],[   0,   0]], # link87、18→5
    [[   0,   0],[9999,9999],[   0,   0],[   0,   0],[   0,   0]], # link88、18→10
    [[   0,   0],[9999,9999],[   0,   0],[   0,   0],[   0,   0]], # link89、19→6
    [[   0,   0],[9999,9999],[   0,   0],[   0,   0],[   0,   0]], # link90、19→11

    [[  ub5,   0],[3*ub5,   0],[  ub5,   0],[  ub5,   0],[   0,  ub5]], # link91、3→3
    [[  ub5,   0],[3*ub5,   0],[  ub5,   0],[  ub5,   0],[   0,  ub5]], # link92、4→4
    [[  ub5,   0],[3*ub5,   0],[  ub5,   0],[  ub5,   0],[   0,  ub5]], # link93、5→5
    [[  ub5,   0],[3*ub5,   0],[  ub5,   0],[  ub5,   0],[   0,  ub5]], # link94、6→6
    [[ usa5,   0],[3*usa5,   0],[ usa5,   0],[ usa5,   0],[   0, usa5]], # link95、8→8
    [[ usa5,   0],[3*usa5,   0],[ usa5,   0],[ usa5,   0],[   0, usa5]], # link96、9→9
    [[ usa5,   0],[3*usa5,   0],[ usa5,   0],[ usa5,   0],[   0, usa5]], # link97、10→10
    [[ usa5,   0],[3*usa5,   0],[ usa5,   0],[ usa5,   0],[   0, usa5]], # link98、11→11
])

#### 目的関数

In [175]:
# フローにかかるコスト（輸送コスト）
# 年ごとに分ける
Flow_cost_list = [m for m in range(N_network)]

# 各年のフローにかかるコスト（輸送コスト）
for m in range(N_network):
    Flow_cost_list[m] = quicksum(c_x[m][i,j,k] * x[m,i,j,k] for i in range(num_link) for j in range(num_product) for k in range(2))

Flow_cost = 0

# 全年のフローにかかるコストを計算（輸送コスト）
for m in range(N_network):
    Flow_cost = Flow_cost + Flow_cost_list[m]



# リンクの使用費
# 年ごとに分ける
Link_opex_list = [m for m in range(N_network)]

# 各年のリンクの使用費
for m in range(N_network):
    Link_opex_list[m] = quicksum(c_s_use[m][i,j,k] * x[m,i,j,k] for i in range(num_link) for j in range(num_product) for k in range(2))

Link_opex = 0

# 全年のリンクの使用費を計算
for m in range(N_network):
    Link_opex = Link_opex + Link_opex_list[m]



# 以下新規項目 将来は変数 cost_ver変数を用いずに表す#####################################################################################################################################
# リンク建設コスト
# ノードの使用開始時にリンク建設コストをかける

# 各年のリンク建設費を変数で表す
Link_capex_list = {}
for m in range(N_network-1):
    # Link_capex_list[m] → m+2年目にかかる拠点建設コスト
    Link_capex_list[m] = model.addVar(vtype="I", name=f"{m+2}年目にかかる拠点建設コスト")

# 2年目以降に建設される可能性があるノードから流出するリンクのNumber
link_ranges = [
    range(47, 56),  # ノード12から流出するリンクのリンク番号が47〜55
    range(56, 65),  # ノード13から流出するリンク
    range(65, 74),  # ノード14から流出するリンク
    range(74, 83)   # ノード15から流出するリンク
]

# 各ノードのm+2年目の建設有無を0-1変数で表現（1で建設、0で建設しない）
# 1度建設されたらそのノードでの再びの建設は考えない
cost_var = {}
for m in range(N_network-1):
    for r in range(len(link_ranges)):
        # m-1 × 建設可能性のある生産拠点の数
        cost_var[m, r] = model.addVar(vtype="B", name=f"{m+2}年目のNode{r+12}のノード建設有無")

# 各年の各ノードから流出するリンクのうちどれかが使われ始めるとcost_varが1になるように制約を追加
for m in range(N_network - 1):
    for r, link_range in enumerate(link_ranges):
        # link_rangesのr番目のリンクのうちどれかが使われ始めるとcost_varが1になる
        model.addCons(cost_var[m, r] >= quicksum(s[m+1, i, 0] - s[m, i, 0] for i in link_range) / len(link_range))

# cost_varが1だった時にコストをかける（年、Nodeごとに値を変える時は書き直しが必要）
for m in range(N_network - 1):
    Link_capex_list[m] = (200 - 50 * m) * quicksum(cost_var[m, r] for r in range(len(link_ranges)))

# 全年の拠点建設費を計算
Link_capex = 0
Link_capex = quicksum(Link_capex_list[m] for m in range(N_network-1))
#####################################################################################################################################



# 目的関数
model.setObjective(Flow_cost + Link_opex + Link_capex, sense = "minimize")

#### 制約条件

In [176]:
# ノードからの流出とノードへの流入の制約
for m in range(0, N_network):
    for l in range(0, num_node):
        for j in range(0, num_product):
            model.addCons(quicksum((s[m,i,0] * A_plus[m][l,i] * x[m,i,j,0] - s[m,i,1] * A_minus[m][l,i] * x[m,i,j,1]) for i in range(num_link)) <= b[m][l,j])

# 輸送するときにplusとminusで同じ値になる
for m in range(0, N_network):
    for aa in range(0, 91): # 輸送リンク
        for j in range(0, num_product):
            model.addCons(x[m,aa,j,0] == x[m,aa,j,1])

# 組立の際の制約
for m in range(0, N_network):
    for bb in range(91, 99): # グラフループ
            model.addCons(3 * x[m,bb,0,0] == x[m,bb,1,0])
            model.addCons(x[m,bb,1,0] == 3 * x[m,bb,2,0])
            model.addCons(x[m,bb,2,0] == x[m,bb,3,0])

# 組立を実行
            model.addCons(x[m,bb,0,0] / 4 + x[m,bb,1,0] / 12 + x[m,bb,2,0] / 4 + x[m,bb,3,0] / 4 == x[m,bb,4,1])
            model.addCons(x[m,bb,0,0] * 0 == x[m,bb,0,1])
            model.addCons(x[m,bb,1,0] * 0 == x[m,bb,1,1])
            model.addCons(x[m,bb,2,0] * 0 == x[m,bb,2,1])
            model.addCons(x[m,bb,3,0] * 0 == x[m,bb,3,1])

# フローがキャパシティを超えないように
for m in range(0, N_network):
    for i in range(0, num_link):
        for j in range(0, num_product):
            for k in range(0, 2):
                model.addCons(x[m,i,j,k] <= u[m][i,j,k])

# sに関する制約条件
# plusとminusで同じ値になる
for m in range(0, N_network):
    for i in range(0, num_link):
        model.addCons(s[m,i,0] == s[m,i,1])

# 後のネットワークのsの方が大きい値をとる
for m in range(0, N_network - 1):
    for i in range(0, num_link):
        model.addCons(s[m + 1,i,0] >= s[m,i,0])

# 1年目からあるリンクが存在するかを設定
for i in range(47, 83):
    for k in range(0, 2):
        model.addCons(s[0,i,k] == 0)

for i in range(83, 99):
    for k in range(0, 2):
        model.addCons(s[0,i,k] == 1)

各設置海域・各年に設置する風車量を決定

In [177]:
# 5年間で4箇所の設置海域に60基の風車を設置する
# 設置海域8（島牧沖）の風車設置量
model.addCons(quicksum(matome[0,m] for m in range(0, N_network)) == 35)

# 設置海域9（銚子沖）の風車設置量
model.addCons(quicksum(matome[1,m] for m in range(0, N_network)) == 35)

# 設置海域10（五島市沖）の風車設置量
model.addCons(quicksum(matome[2,m] for m in range(0, N_network)) == 35)

# 設置海域11（能代沖）の風車設置量
model.addCons(quicksum(matome[3,m] for m in range(0, N_network)) == 35)

c9060

#### 問題を解く

In [178]:
model.setParam("limits/time", 300.0)
# 出力を非表示
model.hideOutput()
model.optimize()


if model.getStatus() == "optimal":
    print("Optimal value:", model.getObjVal())
    print("Solution:")
    # output
    for (m,i,j,k) in x:
        if model.getVal(x[m,i,j,k]) >= 0.5:
            print(f"x[{m},{i},{j},{k}] = {model.getVal(x[m,i,j,k])}")
    
    for (m,i,k) in s:
        if model.getVal(s[m,i,k]) >= 0.5:
            print(f"s[{m},{i},{k}] = {model.getVal(s[m,i,k])}")
    
    for (ins, m) in matome:
        if model.getVal(matome[ins,m]) >= 0.5:
            print(f"設置海域の{ins+8},{m+1}年目 = {model.getVal(matome[ins,m])}")
    
    for (m, r) in cost_var:
        if model.getVal(cost_var[m,r]) >= 0.5:
            print(f"{m+2}年目のNode{r+12}の建設有無 = {model.getVal(cost_var[m,r])}")

Optimal value: 9577.0646
Solution:
x[1,1,2,0] = 1.0
x[1,1,2,1] = 1.0
x[1,2,2,0] = 3.0
x[1,2,2,1] = 3.0
x[1,3,2,0] = 2.0
x[1,3,2,1] = 2.0
x[1,5,2,0] = 2.0
x[1,5,2,1] = 2.0
x[1,6,2,0] = 3.0
x[1,6,2,1] = 3.0
x[1,7,2,0] = 3.0
x[1,7,2,1] = 3.0
x[1,10,3,0] = 1.0
x[1,10,3,1] = 1.0
x[1,11,3,0] = 3.0
x[1,11,3,1] = 3.0
x[1,12,3,0] = 2.0
x[1,12,3,1] = 2.0
x[1,14,3,0] = 2.0
x[1,14,3,1] = 2.0
x[1,15,3,0] = 3.0
x[1,15,3,1] = 3.0
x[1,16,3,0] = 3.0
x[1,16,3,1] = 3.0
x[1,19,0,0] = 1.0
x[1,19,0,1] = 1.0
x[1,20,0,0] = 3.0
x[1,20,0,1] = 3.0
x[1,21,0,0] = 2.0
x[1,21,0,1] = 2.0
x[1,23,0,0] = 2.0
x[1,23,0,1] = 2.0
x[1,24,0,0] = 3.0
x[1,24,0,1] = 3.0
x[1,25,0,0] = 3.0
x[1,25,0,1] = 3.0
x[1,32,4,0] = 1.0
x[1,32,4,1] = 1.0
x[1,37,4,0] = 3.0
x[1,37,4,1] = 3.0
x[1,42,4,0] = 2.0
x[1,42,4,1] = 2.0
x[1,75,1,0] = 3.0
x[1,75,1,1] = 3.0
x[1,76,1,0] = 9.0
x[1,76,1,1] = 9.0
x[1,77,1,0] = 6.0
x[1,77,1,1] = 6.0
x[1,79,1,0] = 6.0
x[1,79,1,1] = 6.0
x[1,80,1,0] = 9.0
x[1,80,1,1] = 9.0
x[1,81,1,0] = 9.0
x[1,81,1,1] = 9.0
x[1,9

#### 結果表示

In [179]:
print(f"輸送コスト = {model.getVal(Flow_cost)}")
print(f"リンク使用コスト = {model.getVal(Link_opex)}")
print(f"リンク建設コスト = {model.getVal(Link_capex)}")
print(f"総コスト = {model.getObjVal()}")

print()

# 各年の輸送コストを表示
for m in range(N_network):
    print(f"{m+1}年目輸送コスト = {model.getVal(Flow_cost_list[m])}")

print()

# 各年のリンク使用コストを表示
for m in range(N_network):
    print(f"{m+1}年目リンク使用コスト = {model.getVal(Link_opex_list[m])}")

print()

# 各年の拠点建設コストを表示
for m in range(N_network-1):
    print(f"{m+2}年目拠点建設コスト = {model.getVal(Link_capex_list[m])}")

print()

# 各年の総コスト
print(f"1年目総コスト = {model.getVal(Flow_cost_list[0]) + model.getVal(Link_opex_list[0])}")

for m in range(N_network-1):
    print(f"{m+2}年目総コスト = {model.getVal(Flow_cost_list[m+1] + Link_opex_list[m+1] + Link_capex_list[m])}")

輸送コスト = 4777.064599999998
リンク使用コスト = 4200.0
リンク建設コスト = 600.0
総コスト = 9577.0646

1年目輸送コスト = 0.0
2年目輸送コスト = 580.006
3年目輸送コスト = 1018.4159
4年目輸送コスト = 1474.4197
5年目輸送コスト = 1704.2230000000002

1年目リンク使用コスト = 0.0
2年目リンク使用コスト = 420.0
3年目リンク使用コスト = 930.0
4年目リンク使用コスト = 1320.0
5年目リンク使用コスト = 1530.0

2年目拠点建設コスト = 200.0
3年目拠点建設コスト = 300.0
4年目拠点建設コスト = 100.0
5年目拠点建設コスト = -2.7755575615628914e-15

1年目総コスト = 0.0
2年目総コスト = 1200.0059999999999
3年目総コスト = 2248.4159
4年目総コスト = 2894.4197000000004
5年目総コスト = 3234.2230000000004


#### 詳しい結果を表示するためのやつ

In [180]:
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [181]:
# indexの作成
# ケースが変わっても使い回しできる（はず）
I = [aaa for aaa in range(num_link * N_network)]

for m in range(N_network):
    for i in range(num_link):
        I[num_link * m + i] = (f'No{i}.ネットワーク{m+1}のLink{i} = {l_taple[i]}')

In [182]:
ppi = pd.DataFrame({'tower':     (model.getVal(x[m,i,0,1]) for m in range(N_network) for i in range(num_link)),
                    'blade':     (model.getVal(x[m,i,1,1]) for m in range(N_network) for i in range(num_link)),
                    'nacelle':   (model.getVal(x[m,i,2,1]) for m in range(N_network) for i in range(num_link)),
                    'foundation':(model.getVal(x[m,i,3,1]) for m in range(N_network) for i in range(num_link)),
                    'product':   (model.getVal(x[m,i,4,1]) for m in range(N_network) for i in range(num_link))},
                    index = I)

In [183]:
ppi.style.set_caption("ケース1")
ppi.loc[(ppi["tower"] >= 0.5) | (ppi["blade"] >= 0.5) | (ppi["nacelle"] >= 0.5) | (ppi["foundation"] >= 0.5) | (ppi["product"] >= 0.5)]

,tower,blade,nacelle,foundation,product
No1.ネットワーク2のLink1 = 0004,-0.0,-0.0,1.0,-0.0,-0.0
No2.ネットワーク2のLink2 = 0005,-0.0,-0.0,3.0,-0.0,-0.0
No3.ネットワーク2のLink3 = 0006,-0.0,-0.0,2.0,-0.0,-0.0
No5.ネットワーク2のLink5 = 0008,-0.0,-0.0,2.0,-0.0,-0.0
No6.ネットワーク2のLink6 = 0009,-0.0,-0.0,3.0,-0.0,-0.0
No7.ネットワーク2のLink7 = 0010,-0.0,-0.0,3.0,-0.0,-0.0
No10.ネットワーク2のLink10 = 0104,-0.0,-0.0,-0.0,1.0,-0.0
No11.ネットワーク2のLink11 = 0105,-0.0,-0.0,-0.0,3.0,-0.0
No12.ネットワーク2のLink12 = 0106,-0.0,-0.0,-0.0,2.0,-0.0
No14.ネットワーク2のLink14 = 0108,-0.0,-0.0,-0.0,2.0,-0.0


In [184]:
# 行列の入れ替えをしたやつ
# 行と列の定義
rows = sorted(set(x[:2] for x in l_taple))  # '00', '01', '02', '12', '13', '14', '15'
columns = sorted(set(x[2:] for x in l_taple))  # '03', '04', '05', '06', '07', '08', '09', '10', '11'

# データを埋めるための実際の関数を使用
def get_val(m, i, j, k):
    # ここで model.getVal を呼び出す
    return model.getVal(x[m, i, j, k])

# データフレームのリストを格納するリスト
dfs = []

# 各年に対してデータフレームを作成
for year in range(1, N_network + 1):
    # l_taple の要素を year 回繰り返す
    l_taple_expanded = l_taple * year

    # DataFrameを作成するための辞書
    data = {f'{row}_{year}年目': [None] * len(columns) for row in rows}

    for m in range(year):
        for i in range(num_link):
            idx = num_link * m + i
            if idx < len(l_taple_expanded):
                row_label = l_taple_expanded[idx][:2]
                col_label = l_taple_expanded[idx][2:]
                
                tower_val = get_val(m, i, 0, 1)
                blade_val = get_val(m, i, 1, 1)
                nacelle_val = get_val(m, i, 2, 1)
                foundation_val = get_val(m, i, 3, 1)
                product_val = get_val(m, i, 4, 1)
                
                # 0 の値を None に置き換える
                tower_val = None if tower_val == 0 else tower_val
                blade_val = None if blade_val == 0 else blade_val
                nacelle_val = None if nacelle_val == 0 else nacelle_val
                foundation_val = None if foundation_val == 0 else foundation_val
                product_val = None if product_val == 0 else product_val
                
                # 行と列のインデックスが有効か確認
                if row_label in rows and col_label in columns:
                    row_idx = rows.index(row_label)
                    col_idx = columns.index(col_label)
                    data[f'{row_label}_{year}年目'][col_idx] = (
                        f"{tower_val:>6}" if tower_val is not None else " ",
                        f"{blade_val:>6}" if blade_val is not None else " ",
                        f"{nacelle_val:>6}" if nacelle_val is not None else " ",
                        f"{foundation_val:>6}" if foundation_val is not None else " ",
                        f"{product_val:>6}" if product_val is not None else " "
                    )

    # DataFrameの作成
    df = pd.DataFrame(data, index=columns)

    # 各セルの要素をフォーマットして整形
    df = df.apply(lambda x: x.map(lambda y: f"({y[0]:>6}, {y[1]:>6}, {y[2]:>6}, {y[3]:>6}, {y[4]:>6})" if y is not None else "None"))

    # タイトルを付けてリストに追加
    dfs.append((f'{year}年目', df))

# 各DataFrameのタイトルと内容を表示
for title, df in dfs:
    print(f'\n{title}のデータフレーム:')
    print(df)


1年目のデータフレーム:
                                      00_1年目  \
03  (      ,       ,       ,       ,       )   
04  (      ,       ,       ,       ,       )   
05  (      ,       ,       ,       ,       )   
06  (      ,       ,       ,       ,       )   
07  (      ,       ,       ,       ,       )   
08  (      ,       ,       ,       ,       )   
09  (      ,       ,       ,       ,       )   
10  (      ,       ,       ,       ,       )   
11  (      ,       ,       ,       ,       )   

                                      01_1年目  \
03  (      ,       ,       ,       ,       )   
04  (      ,       ,       ,       ,       )   
05  (      ,       ,       ,       ,       )   
06  (      ,       ,       ,       ,       )   
07  (      ,       ,       ,       ,       )   
08  (      ,       ,       ,       ,       )   
09  (      ,       ,       ,       ,       )   
10  (      ,       ,       ,       ,       )   
11  (      ,       ,       ,       ,       )   

                        